# CISC 4900 Project

In [173]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [3]:
graph_template = go.layout.Template()
graph_template.layout.plot_bgcolor = 'white'
graph_template.layout.xaxis = dict(
    gridcolor='lightgray'
)
graph_template.layout.yaxis = dict(
    gridcolor='lightgray'
)
pio.templates["graph_template"] = graph_template
pio.templates.default = "graph_template"
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
colors2 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3']

In [4]:
def set_line_fig_labels(fig, fig_title, x_ticks, x_title, y_title, legend=True):
    fig.update_layout(
        title=fig_title,
        xaxis_title=x_title,
        yaxis_title=y_title,
        showlegend=legend,
        xaxis=dict(
            tickvals=x_ticks, 
            ticktext=x_ticks
        )
    )

In [5]:
def set_bar_fig_labels(fig, fig_title, x_title, y_title, legend=True):
    fig.update_layout(
        title=fig_title,
        xaxis_title=x_title,
        yaxis_title=y_title,
        showlegend=legend,
    )

In [6]:
def make_pattern(color, shape="/"):
    pattern=dict(
            shape=shape,
            fillmode="overlay",
            size=12,
            fgcolor="black",  
            bgcolor=color
        )
    return pattern

def make_border():
    line=dict(
        color="black",
        width=2
    )
    return line

In [7]:
graduation_data = pd.read_excel("https://infohub.nyced.org/docs/default-source/default-document-library/2024-graduation-rates-public-school.xlsx", sheet_name=None, engine="openpyxl")
graduation_data.keys()

dict_keys(['NOTES', 'All', 'ELL', 'Ever ELL', 'SWD', 'Ethnicity', 'Gender', 'Poverty'])

In [8]:
general_graduation = graduation_data['All']
ell_graduation = graduation_data['Ever ELL']
swd_graduation = graduation_data['SWD']
ethnicity_graduation = graduation_data['Ethnicity']
poverty_graduation = graduation_data['Poverty']

In [9]:
general_graduation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19901 entries, 0 to 19900
Data columns (total 24 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   DBN                                   19901 non-null  object
 1   School Name                           19901 non-null  object
 2   Category                              19901 non-null  object
 3   Cohort Year                           19901 non-null  int64 
 4   Cohort                                19901 non-null  object
 5   # Total Cohort                        19901 non-null  int64 
 6   # Grads                               19901 non-null  object
 7   % Grads                               19901 non-null  object
 8   # Total Regents                       19901 non-null  object
 9   % Total Regents of Cohort             19901 non-null  object
 10  % Total Regents of Grads              19865 non-null  object
 11  # Advanced Regents          

In [10]:
def clean_df(df):
    df['Graduation Year'] = df['Cohort Year'] + df['Cohort'].str[0].astype(int)
    df['Graduation Month'] = df['Cohort'].str[7:]
    df['Duration'] = df['Cohort'].str[0].astype(int)

    num_columns = ['Cohort Year', '# Total Cohort', '# Grads', '% Grads', '# Total Regents', '% Total Regents of Cohort', '% Total Regents of Grads', '# Advanced Regents', '% Advanced Regents of Cohort', '% Advanced Regents of Grads', '# Regents without Advanced', '% Regents without Advanced of Cohort', '% Regents without Advanced of Grads', '# Local', '% Local of Cohort', '% Local of Grads', '# Still Enrolled', '% Still Enrolled', '# Dropout', '% Dropout']

    for column in num_columns:
        if column[0] == '%':
            df = df.drop(column, axis=1)
        else:
            df[column] = df[column].replace('s', np.nan)
            df[column] = pd.to_numeric(df[column], errors="coerce")
    
    return df

In [11]:
general_graduation = clean_df(general_graduation)
ell_graduation = clean_df(ell_graduation)
swd_graduation = clean_df(swd_graduation)
poverty_graduation = clean_df(poverty_graduation)
ethnicity_graduation = clean_df(ethnicity_graduation)

In [12]:
general_graduation = general_graduation[general_graduation['Graduation Year'] > 2017]
swd_graduation = swd_graduation[swd_graduation['Graduation Year'] > 2017]
ethnicity_graduation = ethnicity_graduation[ethnicity_graduation['Graduation Year'] > 2017]
ell_graduation = ell_graduation[ell_graduation['Graduation Year'] > 2017]
poverty_graduation = poverty_graduation[poverty_graduation['Graduation Year'] > 2017]

In [13]:
def filter_data(df, columns, cluster=False):
    agg_dict = {}
    for column in columns:
        agg_dict[column] = 'max'
    agg_dict.update({
        '# Total Cohort' : 'first',
        'Graduation Month' : 'first'
    })
    if cluster:
        filtered_data = df.groupby(['School Name', 'Category', 'Graduation Year', 'Duration', 'Cluster']).agg(agg_dict).reset_index()
    else:
        filtered_data = df.groupby(['School Name', 'Category', 'Graduation Year', 'Duration']).agg(agg_dict).reset_index()

    return filtered_data

In [14]:
test_results = pd.read_excel("https://infohub.nyced.org/docs/default-source/default-document-library/2014-15-to-2022-23-nyc-regents-overall-and-by-category---public.xlsx", sheet_name=None, engine="openpyxl")
test_results.keys()

/Users/mwumigue/.pyenv/versions/3.10.0/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


dict_keys(['Notes', 'All Students', 'By Gender', 'By Ethnicity', 'By ELL Status', 'By SWD Status', 'School Profile', 'school list'])

In [194]:
general_test_results = test_results['All Students']
ell_test_results = test_results['By ELL Status']
swd_test_results = test_results['By SWD Status']
ethnicity_test_results = test_results['By Ethnicity']

In [16]:
def classify_subject(row):
    regents = row['Regents Exam']

    English = ['Common Core English', 'English']
    Mathematics = ['Algebra2/Trigonometry', 'Common Core Algebra', 'Common Core Geometry', 'Geometry', 'Integrated Algebra', 'Common Core Algebra2']
    Science = ['Living Environment', 'Physical Settings/Chemistry', 'Physical Settings/Earth Science', 'Physical Settings/Physics']
    Social_Studies = ['Global History and Geography', 'US History and Government']
    Languages_Other_Than_English = ['Chinese', 'Spanish', 'Italian', 'French']

    if regents in English:
        return "English"
    if regents in Mathematics:
        return "Mathematics"
    if regents in Science:
        return "Science"
    if regents in Social_Studies:
        return "Social Studies"
    if regents in Languages_Other_Than_English:
        return "Languages Other Than English"

In [17]:
school_list = general_graduation['School Name'].unique().tolist()

In [18]:
def filter_regent_scores(df):
    new_df = df[df['School Name'].str.upper().isin(school_list)].reset_index()
    new_df = new_df[['School Name', 'Year', 'Category', 'Regents Exam', 'Total Tested', 'Number Scoring 65 or Above']]
    new_df = new_df[~new_df['Year'].isin([2020, 2021])]
    new_df['Regents Subject'] = new_df.apply(classify_subject, axis=1)
    num_columns = ['Total Tested', 'Number Scoring 65 or Above']
    for column in num_columns:
        new_df[column] = new_df[column].replace('s', np.nan)
        new_df[column] = pd.to_numeric(new_df[column], errors="coerce")
    return new_df

In [195]:
general_test_results = filter_regent_scores(general_test_results)
ell_test_results = filter_regent_scores(ell_test_results)
swd_test_results = filter_regent_scores(swd_test_results)
ethnicity_test_results = filter_regent_scores(ethnicity_test_results)

## Graduation Summary

In [20]:
all_graduates_general = filter_data(general_graduation, ['# Grads','# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout'])
all_graduates_general

,School Name,Category,Graduation Year,Duration,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,# Total Cohort,Graduation Month
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2018,4,30.0,7.0,20.0,3.0,3.0,2.0,35,August
1,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2018,5,33.0,4.0,25.0,4.0,0.0,1.0,35,August
2,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2018,6,54.0,8.0,37.0,9.0,0.0,2.0,58,June
3,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2019,4,37.0,6.0,23.0,8.0,0.0,3.0,40,August
4,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2019,5,32.0,7.0,21.0,4.0,1.0,2.0,35,August
...,...,...,...,...,...,...,...,...,...,...,...,...
9948,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2023,5,76.0,63.0,13.0,0.0,0.0,1.0,77,August
9949,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2023,6,62.0,53.0,9.0,0.0,2.0,2.0,66,June
9950,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2024,4,71.0,51.0,20.0,0.0,2.0,0.0,73,August
9951,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2024,5,75.0,27.0,48.0,0.0,0.0,0.0,75,August


### Population

In [21]:
def filter_graduation_data(df, columns, cluster=False):
    agg_dict = {}
    for column in ['# Total Cohort', '# Grads', '# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout']:
        agg_dict[column] = 'sum'
    if cluster == True:
        columns.append('Cluster')
    filtered_df = df.groupby(columns).agg(agg_dict).reset_index()
    return filtered_df

In [22]:
general_data = filter_graduation_data(all_graduates_general,columns=['Graduation Year'])
general_data

,Graduation Year,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout
0,2018,213338,172686.0,44135.0,113986.0,14566.0,18562.0,21841.0
1,2019,213275,175297.0,45937.0,113030.0,16332.0,15493.0,22491.0
2,2020,214042,180304.0,53991.0,113656.0,12660.0,13120.0,19148.0
3,2021,211698,182263.0,65114.0,110306.0,6955.0,12820.0,15854.0
4,2022,208741,182956.0,74240.0,107180.0,1543.0,9625.0,15197.0
5,2023,204843,181402.0,70198.0,110469.0,782.0,9032.0,13941.0
6,2024,201475,178675.0,59297.0,117140.0,2261.0,9645.0,13129.0


In [23]:
general_data_population = general_data[['Graduation Year', '# Total Cohort', '# Grads']].copy()
general_data_population['Grad %'] = general_data_population['# Grads'] / general_data_population['# Total Cohort'] * 100
general_data_population

,Graduation Year,# Total Cohort,# Grads,Grad %
0,2018,213338,172686.0,80.944792
1,2019,213275,175297.0,82.192943
2,2020,214042,180304.0,84.237673
3,2021,211698,182263.0,86.095759
4,2022,208741,182956.0,87.647372
5,2023,204843,181402.0,88.556602
6,2024,201475,178675.0,88.683459


In [24]:
fig = go.Figure()
fig.add_trace(go.Bar(x=general_data_population['Graduation Year'], y=general_data_population['# Total Cohort'], marker=dict(color=colors[0]), name='Total Population'))
fig.add_trace(go.Bar(x=general_data_population['Graduation Year'], y=general_data_population['# Grads'], name='Graduate Population', marker=dict(color=colors[0], pattern=make_pattern(colors[0]), line=make_border())))

fig.update_layout(
    xaxis=dict(
        tickvals=general_data_population['Graduation Year'],
        ticktext=general_data_population['Graduation Year']
    ),
    barmode='overlay'
)

set_bar_fig_labels(fig=fig, fig_title='Graduate Population from 2018 to 2024', x_title='Year', y_title='Population')
fig.show()

### Rates

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=general_data_population['Graduation Year'], y=general_data_population['Grad %']))
set_line_fig_labels(fig, fig_title="Graduation Rate from 2018 to 2024", x_title="Year", y_title="Graduation %", x_ticks=general_data_population['Graduation Year'], legend=False)
fig.sh

### Diploma Type

In [26]:
general_data_diploma_type = general_data[['Graduation Year', '# Advanced Regents', '# Regents without Advanced', '# Local']].copy()
general_data_diploma_type

,Graduation Year,# Advanced Regents,# Regents without Advanced,# Local
0,2018,44135.0,113986.0,14566.0
1,2019,45937.0,113030.0,16332.0
2,2020,53991.0,113656.0,12660.0
3,2021,65114.0,110306.0,6955.0
4,2022,74240.0,107180.0,1543.0
5,2023,70198.0,110469.0,782.0
6,2024,59297.0,117140.0,2261.0


In [27]:
fig = go.Figure()

fig.add_trace(go.Bar(x=general_data_diploma_type['Graduation Year'], y=general_data_diploma_type['# Advanced Regents'], name='Advanced Regents Diploma'))
fig.add_trace(go.Bar(x=general_data_diploma_type['Graduation Year'], y=general_data_diploma_type['# Regents without Advanced'], name='Regular Diploma'))
fig.add_trace(go.Bar(x=general_data_diploma_type['Graduation Year'], y=general_data_diploma_type['# Local'], name='Local Diploma'))

fig.update_layout(
    xaxis=dict(
        tickvals=general_data_diploma_type['Graduation Year'].unique(), 
        ticktext=general_data_diploma_type['Graduation Year'].unique()
    )
)
set_bar_fig_labels(fig=fig, fig_title='Distribution of Graduates Based on Diploma Type', y_title='Population', x_title='Year')

fig.show()

### Dropouts

In [28]:
general_data_dropout = general_data[['Graduation Year', '# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']].copy()
general_data_dropout['# No Grads'] = general_data_dropout['# Total Cohort'] - general_data_dropout['# Grads']
general_data_dropout['Dropout %'] = general_data_dropout['# Dropout'] / general_data_dropout['# No Grads'] * 100
general_data_dropout

,Graduation Year,# Total Cohort,# Grads,# Dropout,# Still Enrolled,# No Grads,Dropout %
0,2018,213338,172686.0,21841.0,18562.0,40652.0,53.726754
1,2019,213275,175297.0,22491.0,15493.0,37978.0,59.221128
2,2020,214042,180304.0,19148.0,13120.0,33738.0,56.754994
3,2021,211698,182263.0,15854.0,12820.0,29435.0,53.861050
4,2022,208741,182956.0,15197.0,9625.0,25785.0,58.937367
5,2023,204843,181402.0,13941.0,9032.0,23441.0,59.472719
6,2024,201475,178675.0,13129.0,9645.0,22800.0,57.583333


In [29]:
fig = go.Figure()

fig.add_trace(go.Bar(x=general_data_dropout['Graduation Year'], y=general_data_dropout['# Still Enrolled'], name='Still Enrolled'))
fig.add_trace(go.Bar(x=general_data_dropout['Graduation Year'], y=general_data_dropout['# Dropout'], name='Dropout'))
fig.update_layout(
    xaxis=dict(
        tickvals=general_data_dropout['Graduation Year'].unique(), 
        ticktext=general_data_dropout['Graduation Year'].unique()
    )
)
set_bar_fig_labels(fig=fig, fig_title='Distribution of Non-Graduates from 2018 to 2024', y_title='Population', x_title='Year')

fig.show()

### Test Results

In [30]:
general_test_results

,School Name,Year,Category,Regents Exam,Total Tested,Number Scoring 65 or Above,Regents Subject
0,Orchard Collegiate Academy,2015,All Students,Algebra2/Trigonometry,5,NaN,Mathematics
1,Orchard Collegiate Academy,2015,All Students,Chinese,1,NaN,Languages Other Than English
2,Orchard Collegiate Academy,2015,All Students,Common Core Algebra,12,1.0,Mathematics
3,Orchard Collegiate Academy,2015,All Students,Common Core English,34,19.0,English
4,Orchard Collegiate Academy,2015,All Students,Common Core Geometry,10,1.0,Mathematics
...,...,...,...,...,...,...,...
31069,Bushwick Community High School,2023,All Students,Common Core Algebra,27,12.0,Mathematics
31070,Bushwick Community High School,2023,All Students,Common Core English,42,21.0,English
31071,Bushwick Community High School,2023,All Students,Global History and Geography,29,12.0,Social Studies
31072,Bushwick Community High School,2023,All Students,Living Environment,12,5.0,Science


In [31]:
all_graduates_general_test_results = general_test_results.groupby(['Regents Subject'])[['Total Tested', 'Number Scoring 65 or Above']].sum().reset_index()
all_graduates_general_test_results['% Passed'] = all_graduates_general_test_results['Number Scoring 65 or Above'] / all_graduates_general_test_results['Total Tested'] * 100
all_graduates_general_test_results

,Regents Subject,Total Tested,Number Scoring 65 or Above,% Passed
0,English,508598,387540.0,76.197704
1,Languages Other Than English,156096,141239.0,90.482139
2,Mathematics,1096353,612471.0,55.864398
3,Science,868837,524015.0,60.312233
4,Social Studies,831037,571437.0,68.761920


In [224]:
fig = go.Figure()

for subject in all_graduates_general_test_results['Regents Subject'].unique():
    subject_row = all_graduates_general_test_results[all_graduates_general_test_results['Regents Subject'] == subject]
    subject_index = all_graduates_general_test_results['Regents Subject'].unique().tolist().index(subject)
    fig.add_trace(go.Bar(x=subject_row['Regents Subject'], y=subject_row['% Passed'], showlegend=False, marker=dict(color=colors2[subject_index])))
    fig.update_yaxes(title_text='% of Approved Students')

fig.update_layout(
    title_text='Rate of Students Who Approved Each Regents Subject'
)
fig.show()

In [33]:
all_graduates_general_test_results_per_year = general_test_results.groupby(['Year','Regents Subject'])[['Total Tested', 'Number Scoring 65 or Above']].sum().reset_index()
all_graduates_general_test_results_per_year['% Passed'] = all_graduates_general_test_results_per_year['Number Scoring 65 or Above'] / all_graduates_general_test_results_per_year['Total Tested'] * 100
all_graduates_general_test_results_per_year

,Year,Regents Subject,Total Tested,Number Scoring 65 or Above,% Passed
0,2015,English,91014,69097.0,75.919089
1,2015,Languages Other Than English,21783,20956.0,96.203461
2,2015,Mathematics,205267,111683.0,54.408648
3,2015,Science,132758,85344.0,64.285391
4,2015,Social Studies,141201,96315.0,68.211273
5,2016,English,76770,58136.0,75.727498
6,2016,Languages Other Than English,21274,19624.0,92.244054
7,2016,Mathematics,183412,101998.0,55.611410
8,2016,Science,133520,85403.0,63.962702
9,2016,Social Studies,139544,93135.0,66.742389


In [226]:
fig = go.Figure()

for subject in all_graduates_general_test_results_per_year['Regents Subject'].unique():
    subject_row = all_graduates_general_test_results_per_year[all_graduates_general_test_results_per_year['Regents Subject'] == subject]
    subject_index = all_graduates_general_test_results_per_year['Regents Subject'].unique().tolist().index(subject)
    fig.add_trace(go.Scatter(x=subject_row['Year'], y=subject_row['% Passed'], name=subject, marker=dict(color=colors2[subject_index])))
    fig.update_yaxes(title_text='% of Approved Students')

fig.update_layout(title_text='Rate of Students Who Approved Each Regents Subject from 2015 to 2023')
fig.show()

## Impact of Duration

In [35]:
general_duration_data = filter_graduation_data(all_graduates_general, ['Graduation Year', 'Duration'])
general_duration_data

,Graduation Year,Duration,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout
0,2018,4,72096,56788.0,15653.0,36436.0,4700.0,11779.0,5119.0
1,2018,5,70207,57281.0,14653.0,37597.0,5031.0,4497.0,7712.0
2,2018,6,71035,58617.0,13829.0,39953.0,4835.0,2286.0,9010.0
3,2019,4,71107,56901.0,15595.0,36177.0,5131.0,10577.0,5319.0
4,2019,5,71984,59937.0,15682.0,38508.0,5747.0,3050.0,8485.0
5,2019,6,70184,58459.0,14660.0,38345.0,5454.0,1866.0,8687.0
6,2020,4,71155,58581.0,22655.0,34976.0,952.0,8944.0,4010.0
7,2020,5,70956,60494.0,15649.0,39162.0,5684.0,2907.0,6795.0
8,2020,6,71931,61229.0,15687.0,39518.0,6024.0,1269.0,8343.0
9,2021,4,69646,59468.0,26725.0,32677.0,178.0,8173.0,3207.0


### Correlation

In [137]:
def correlation_matrix(df):
    fig = px.imshow(
        df.corr(),
        text_auto=True,
        color_continuous_scale='RdBu_r',
        width=1000,
        height=800,
    )
    fig.update_traces(textfont=dict(size=10))
    fig.update_layout(margin=dict(l=125, b=125))
    fig.show()

In [138]:
correlation_matrix(general_duration_data)

### Graduation Rates

In [38]:
general_duration_data_rates = general_duration_data[['Graduation Year', 'Duration', '# Total Cohort', '# Grads']].copy()
general_duration_data_rates['Grad %'] = general_duration_data_rates['# Grads'] / general_duration_data_rates['# Total Cohort'] * 100
general_duration_data_rates

,Graduation Year,Duration,# Total Cohort,# Grads,Grad %
0,2018,4,72096,56788.0,78.767199
1,2018,5,70207,57281.0,81.588730
2,2018,6,71035,58617.0,82.518477
3,2019,4,71107,56901.0,80.021658
4,2019,5,71984,59937.0,83.264337
5,2019,6,70184,58459.0,83.293913
6,2020,4,71155,58581.0,82.328719
7,2020,5,70956,60494.0,85.255651
8,2020,6,71931,61229.0,85.121853
9,2021,4,69646,59468.0,85.386095


In [39]:
fig = go.Figure()

for duration in general_duration_data_rates['Duration'].unique():
    duration_row = general_duration_data_rates[general_duration_data_rates['Duration'] == duration]
    fig.add_trace(go.Scatter(x=duration_row['Graduation Year'], y=duration_row['Grad %'], name=f'{duration} Years'))

set_line_fig_labels(fig=fig, fig_title="Graduation Rate Based on Duration", x_ticks=general_duration_data_rates['Graduation Year'], x_title='Year', y_title='Graduation %')
fig.show()

### Dropouts

In [40]:
general_duration_data_dropout = general_duration_data[['Graduation Year', 'Duration', '# Total Cohort', '# Grads', '# Dropout']].copy()
general_duration_data_dropout['# No Grads'] = general_duration_data_dropout['# Total Cohort'] - general_duration_data_dropout['# Grads']
general_duration_data_dropout['Dropout %'] = general_duration_data_dropout['# Dropout'] / general_duration_data_dropout['# No Grads'] * 100
general_duration_data_dropout

,Graduation Year,Duration,# Total Cohort,# Grads,# Dropout,# No Grads,Dropout %
0,2018,4,72096,56788.0,5119.0,15308.0,33.440031
1,2018,5,70207,57281.0,7712.0,12926.0,59.662695
2,2018,6,71035,58617.0,9010.0,12418.0,72.555967
3,2019,4,71107,56901.0,5319.0,14206.0,37.441926
4,2019,5,71984,59937.0,8485.0,12047.0,70.432473
5,2019,6,70184,58459.0,8687.0,11725.0,74.089552
6,2020,4,71155,58581.0,4010.0,12574.0,31.891204
7,2020,5,70956,60494.0,6795.0,10462.0,64.949340
8,2020,6,71931,61229.0,8343.0,10702.0,77.957391
9,2021,4,69646,59468.0,3207.0,10178.0,31.509137


In [41]:
fig = go.Figure()
for duration in general_duration_data_dropout['Duration'].unique():
    duration_row = general_duration_data_dropout[general_duration_data_dropout['Duration'] == duration]
    fig.add_trace(go.Scatter(x=duration_row['Graduation Year'], y=duration_row['Dropout %'], name=f'{duration} Years'))

set_line_fig_labels(fig=fig, fig_title='Dropout Rates Based on Duration', x_ticks=general_duration_data_dropout['Graduation Year'], x_title='Year', y_title='Dropout %')

fig.show()

## Clusters

In [42]:
from sklearn.cluster import KMeans

In [43]:
classify_data = all_graduates_general[all_graduates_general['Graduation Year'] == 2018]
classify_data = classify_data.groupby(['School Name','Graduation Year']).agg({
    '# Grads' : 'sum',
    '# Total Cohort' : 'sum',
}).reset_index()
classify_data['% Grad'] = classify_data['# Grads'] / classify_data['# Total Cohort'] * 100
classify_data

,School Name,Graduation Year,# Grads,# Total Cohort,% Grad
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,2018,117.0,128,91.406250
1,A-TECH HIGH SCHOOL,2018,188.0,254,74.015748
2,A. PHILIP RANDOLPH CAMPUS HIGH SCHOOL,2018,709.0,879,80.659841
3,ABRAHAM LINCOLN HIGH SCHOOL,2018,1113.0,1445,77.024221
4,ACADEMY FOR CAREERS IN TELEVISION AND FILM,2018,386.0,390,98.974359
...,...,...,...,...,...
484,YORK EARLY COLLEGE ACADEMY,2018,242.0,245,98.775510
485,YOUNG WOMEN'S LEADERSHIP SCHOOL,2018,239.0,239,100.000000
486,YOUNG WOMEN'S LEADERSHIP SCHOOL OF BROOKLYN,2018,181.0,183,98.907104
487,"YOUNG WOMEN'S LEADERSHIP SCHOOL, ASTORIA",2018,232.0,235,98.723404


In [44]:
X = classify_data['% Grad'].values.reshape(-1,1)
X 

array([[ 91.40625   ],
       [ 74.01574803],
       [ 80.65984073],
       [ 77.02422145],
       [ 98.97435897],
       [ 70.77625571],
       [ 83.62831858],
       [ 76.69172932],
       [ 83.5443038 ],
       [ 95.53571429],
       [ 74.03508772],
       [ 48.27586207],
       [ 91.42011834],
       [ 92.49011858],
       [ 96.15931721],
       [ 95.39641944],
       [ 72.78481013],
       [ 83.45864662],
       [ 79.56989247],
       [ 77.01149425],
       [ 78.57142857],
       [ 99.27536232],
       [ 60.33519553],
       [ 72.24334601],
       [ 88.8568684 ],
       [ 72.54901961],
       [ 31.74061433],
       [ 54.32098765],
       [ 81.48148148],
       [ 77.46478873],
       [ 66.07142857],
       [ 84.76070529],
       [ 99.58506224],
       [ 99.47089947],
       [ 99.77578475],
       [ 99.70149254],
       [ 98.52882704],
       [ 99.17098446],
       [ 99.23664122],
       [ 51.32743363],
       [ 93.47079038],
       [ 96.42857143],
       [ 79.20792079],
       [ 91

In [231]:
inertias = []
ks = []
for k in range(1, 11):
    ks.append(k)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

fig = go.Figure()
fig.add_trace(go.Scatter(x=ks, y=inertias))
fig.update_yaxes(title_text='Inertia')
fig.update_xaxes(title_text='Number of Clusters')
fig.update_layout(
    width=600,
    height=500,
    xaxis=dict(
        tickvals=ks, 
        ticktext=ks,
    ),
    title='Inertia Based on Number of Clusters'
) 

In [46]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(X)
classify_data['Cluster'] = kmeans.labels_

In [45]:
classify_data[['School Name', 'Cluster']]

,School Name,Cluster
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,2
1,A-TECH HIGH SCHOOL,1
2,A. PHILIP RANDOLPH CAMPUS HIGH SCHOOL,1
3,ABRAHAM LINCOLN HIGH SCHOOL,1
4,ACADEMY FOR CAREERS IN TELEVISION AND FILM,2
...,...,...
484,YORK EARLY COLLEGE ACADEMY,2
485,YOUNG WOMEN'S LEADERSHIP SCHOOL,2
486,YOUNG WOMEN'S LEADERSHIP SCHOOL OF BROOKLYN,2
487,"YOUNG WOMEN'S LEADERSHIP SCHOOL, ASTORIA",2


In [47]:
general_graduation = classify_data[['School Name', 'Cluster']].merge(general_graduation, on='School Name')
swd_graduation = classify_data[['School Name', 'Cluster']].merge(swd_graduation, on='School Name')
poverty_graduation = classify_data[['School Name', 'Cluster']].merge(poverty_graduation, on='School Name')
ell_graduation = classify_data[['School Name', 'Cluster']].merge(ell_graduation, on='School Name')
ethnicity_graduation = classify_data[['School Name', 'Cluster']].merge(ethnicity_graduation, on='School Name')

In [48]:
general_test_results['School Name'] = general_test_results['School Name'].str.upper()
swd_test_results['School Name'] = swd_test_results['School Name'].str.upper()
ell_test_results['School Name'] = ell_test_results['School Name'].str.upper()
ethnicity_test_results['School Name'] = ethnicity_test_results['School Name'].str.upper()
general_test_results

,School Name,Year,Category,Regents Exam,Total Tested,Number Scoring 65 or Above,Regents Subject
0,ORCHARD COLLEGIATE ACADEMY,2015,All Students,Algebra2/Trigonometry,5,NaN,Mathematics
1,ORCHARD COLLEGIATE ACADEMY,2015,All Students,Chinese,1,NaN,Languages Other Than English
2,ORCHARD COLLEGIATE ACADEMY,2015,All Students,Common Core Algebra,12,1.0,Mathematics
3,ORCHARD COLLEGIATE ACADEMY,2015,All Students,Common Core English,34,19.0,English
4,ORCHARD COLLEGIATE ACADEMY,2015,All Students,Common Core Geometry,10,1.0,Mathematics
...,...,...,...,...,...,...,...
31069,BUSHWICK COMMUNITY HIGH SCHOOL,2023,All Students,Common Core Algebra,27,12.0,Mathematics
31070,BUSHWICK COMMUNITY HIGH SCHOOL,2023,All Students,Common Core English,42,21.0,English
31071,BUSHWICK COMMUNITY HIGH SCHOOL,2023,All Students,Global History and Geography,29,12.0,Social Studies
31072,BUSHWICK COMMUNITY HIGH SCHOOL,2023,All Students,Living Environment,12,5.0,Science


In [49]:
general_test_results['School Name'] = general_test_results['School Name'].str.upper()
swd_test_results['School Name'] = swd_test_results['School Name'].str.upper()
ell_test_results['School Name'] = ell_test_results['School Name'].str.upper()
ethnicity_test_results['School Name'] = ethnicity_test_results['School Name'].str.upper()

general_test_results = classify_data[['School Name', 'Cluster']].merge(general_test_results, on='School Name')
swd_test_results = classify_data[['School Name', 'Cluster']].merge(swd_test_results, on='School Name')
ell_test_results = classify_data[['School Name', 'Cluster']].merge(ell_test_results, on='School Name')
ethnicity_test_results = classify_data[['School Name', 'Cluster']].merge(ethnicity_test_results, on='School Name')

In [50]:
all_graduates_general_cluster = filter_data(general_graduation, ['# Grads','# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout'], cluster=True)
all_graduates_general_cluster

,School Name,Category,Graduation Year,Duration,Cluster,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,# Total Cohort,Graduation Month
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2018,4,2,30.0,7.0,20.0,3.0,3.0,2.0,35,August
1,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2018,5,2,33.0,4.0,25.0,4.0,0.0,1.0,35,August
2,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2018,6,2,54.0,8.0,37.0,9.0,0.0,2.0,58,June
3,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2019,4,2,37.0,6.0,23.0,8.0,0.0,3.0,40,August
4,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,All Students,2019,5,2,32.0,7.0,21.0,4.0,1.0,2.0,35,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9860,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2023,5,2,76.0,63.0,13.0,0.0,0.0,1.0,77,August
9861,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2023,6,2,62.0,53.0,9.0,0.0,2.0,2.0,66,June
9862,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2024,4,2,71.0,51.0,20.0,0.0,2.0,0.0,73,August
9863,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",All Students,2024,5,2,75.0,27.0,48.0,0.0,0.0,0.0,75,August


### Correlation Matrix

In [51]:
general_data_cluster = filter_graduation_data(all_graduates_general_cluster,['Graduation Year', 'Duration'], cluster=True)
general_data_cluster

,Graduation Year,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout
0,2018,4,0,5003,1169.0,56.0,928.0,185.0,2882.0,969.0
1,2018,4,1,31418,22878.0,2475.0,17593.0,2810.0,6027.0,3362.0
2,2018,4,2,35675,32741.0,13122.0,17915.0,1705.0,2870.0,788.0
3,2018,5,0,5557,2130.0,62.0,1613.0,455.0,1698.0,1687.0
4,2018,5,1,30233,22997.0,2428.0,17599.0,2970.0,2015.0,4870.0
...,...,...,...,...,...,...,...,...,...,...
58,2024,5,1,27408,23879.0,4055.0,19665.0,161.0,523.0,2732.0
59,2024,5,2,35000,33632.0,14518.0,19019.0,95.0,277.0,915.0
60,2024,6,0,3809,2145.0,50.0,1997.0,98.0,271.0,1246.0
61,2024,6,1,28123,24626.0,6542.0,17952.0,132.0,210.0,2942.0


In [139]:
correlation_matrix(general_data_cluster)

### Population

In [53]:
general_data_cluster_population = general_data_cluster.groupby('Cluster')[['# Total Cohort', '# Grads']].sum().reset_index()
general_data_cluster_population

,Cluster,# Total Cohort,# Grads
0,0,102667,43625.0
1,1,619211,504589.0
2,2,739952,700591.0


In [54]:
fig = go.Figure()

for cluster in general_data_cluster_population['Cluster'].unique():
    cluster_row = general_data_cluster_population[general_data_cluster_population['Cluster'] == cluster]
    cluster_index = general_data_cluster_population['Cluster'].unique().tolist().index(cluster)
    fig.add_trace(go.Bar(x=cluster_row['Cluster'], y=cluster_row['# Total Cohort'], name='Total Population', marker=dict(color=colors[cluster_index])))
    fig.add_trace(go.Bar(x=cluster_row['Cluster'], y=cluster_row['# Grads'], name='Graduate Population', marker=dict(color=colors[cluster_index], pattern=make_pattern(colors[cluster_index]), line=make_border())))

fig.update_layout(
    barmode='overlay',
        xaxis=dict(
        tickvals=general_data_cluster_population['Cluster'].unique(), 
        ticktext=general_data_cluster_population['Cluster'].unique()
    ),
)

for i in range(len(fig.data) - len(['# Total Cohort', '# Grads'])):
    fig.data[i].showlegend = False
    
set_bar_fig_labels(fig=fig, fig_title='Cluster Population', y_title='Population', x_title='Cluster')

fig.show()

### Graduation Rate

In [55]:
general_data_cluster_rate = general_data_cluster.groupby(['Cluster', 'Graduation Year'])[['# Total Cohort', '# Grads']].sum().reset_index()
general_data_cluster_rate['Grad %'] = general_data_cluster_rate['# Grads'] / general_data_cluster_rate['# Total Cohort'] * 100
general_data_cluster_rate

,Cluster,Graduation Year,# Total Cohort,# Grads,Grad %
0,0,2018,16531,6148.0,37.190733
1,0,2019,16447,6224.0,37.842768
2,0,2020,16496,6795.0,41.191804
3,0,2021,15057,6770.0,44.962476
4,0,2022,13333,6214.0,46.606165
5,0,2023,12250,5578.0,45.534694
6,0,2024,12553,5896.0,46.968852
7,1,2018,92602,69497.0,75.049135
8,1,2019,91860,71002.0,77.293708
9,1,2020,91519,73368.0,80.166960


In [56]:
fig = go.Figure()
for cluster in general_data_cluster_rate['Cluster'].unique():
    cluster_row = general_data_cluster_rate[general_data_cluster_rate['Cluster'] == cluster]
    fig.add_trace(go.Scatter(x=cluster_row['Graduation Year'], y=cluster_row['Grad %'], name=f'Cluster {cluster}'))
set_line_fig_labels(fig=fig, fig_title="Graduation Rate Per Cluster from 2018 to 2024", x_title='Year', y_title="Graduation %", x_ticks=general_data_cluster_rate['Graduation Year'])
fig.show()

### Diploma Type

In [57]:
general_data_cluster_diploma_type = general_data_cluster.groupby(['Cluster'])[['# Advanced Regents', '# Regents without Advanced', '# Local', '# Grads']].sum().reset_index()
general_data_cluster_diploma_type['Advanced Regents %'] = general_data_cluster_diploma_type['# Advanced Regents'] / general_data_cluster_diploma_type['# Grads'] * 100
general_data_cluster_diploma_type['Regents without Advanced %'] = general_data_cluster_diploma_type['# Regents without Advanced'] / general_data_cluster_diploma_type['# Grads'] * 100
general_data_cluster_diploma_type['Local %'] = general_data_cluster_diploma_type['# Local'] / general_data_cluster_diploma_type['# Grads'] * 100
general_data_cluster_diploma_type

,Cluster,# Advanced Regents,# Regents without Advanced,# Local,# Grads,Advanced Regents %,Regents without Advanced %,Local %
0,0,1330.0,37079.0,5223.0,43625.0,3.048711,84.994842,11.972493
1,1,89342.0,383928.0,31379.0,504589.0,17.705895,76.087271,6.218725
2,2,321332.0,361002.0,18385.0,700591.0,45.865848,51.528210,2.624213


In [58]:
fig = make_subplots(rows=1, cols=len(general_data_cluster_diploma_type['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in general_data_cluster_diploma_type['Cluster'].unique():
    cluster_row = general_data_cluster_diploma_type[general_data_cluster_diploma_type['Cluster'] == cluster]
    cluster_index = general_data_cluster_diploma_type['Cluster'].unique().tolist().index(cluster)
    fig.add_trace(go.Bar(x=cluster_row['Cluster'], y=cluster_row['Advanced Regents %'], name=f'Advanced Regents Diploma', marker=dict(color=colors[0])), row=1, col=cluster_index+1)
    fig.add_trace(go.Bar(x=cluster_row['Cluster'], y=cluster_row['Regents without Advanced %'], name=f'Regular Regents Diploma', marker=dict(color=colors[1])), row=1, col=cluster_index+1)
    fig.add_trace(go.Bar(x=cluster_row['Cluster'], y=cluster_row['Local %'], name=f'Local Diploma', marker=dict(color=colors[2])), row=1, col=cluster_index+1)
    fig.update_yaxes(title_text='Distribution %', range=[0,max(general_data_cluster_diploma_type[['Advanced Regents %', 'Regents without Advanced %', 'Local %']].max(axis=1))])

for i in range(len(fig.data) - len(colors)):
    fig.data[i].showlegend = False

fig.update_layout(title_text = "Graduate Distribution in Every Cluster based on Diploma Type")
fig.show()

### Dropout Rate

In [59]:
general_data_cluster

,Graduation Year,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout
0,2018,4,0,5003,1169.0,56.0,928.0,185.0,2882.0,969.0
1,2018,4,1,31418,22878.0,2475.0,17593.0,2810.0,6027.0,3362.0
2,2018,4,2,35675,32741.0,13122.0,17915.0,1705.0,2870.0,788.0
3,2018,5,0,5557,2130.0,62.0,1613.0,455.0,1698.0,1687.0
4,2018,5,1,30233,22997.0,2428.0,17599.0,2970.0,2015.0,4870.0
...,...,...,...,...,...,...,...,...,...,...
58,2024,5,1,27408,23879.0,4055.0,19665.0,161.0,523.0,2732.0
59,2024,5,2,35000,33632.0,14518.0,19019.0,95.0,277.0,915.0
60,2024,6,0,3809,2145.0,50.0,1997.0,98.0,271.0,1246.0
61,2024,6,1,28123,24626.0,6542.0,17952.0,132.0,210.0,2942.0


In [206]:
general_data_cluster_dropout_rate = general_data_cluster[['Graduation Year', 'Duration', 'Cluster', '# Dropout', '# Grads', '# Total Cohort']].copy()
general_data_cluster_dropout_rate['# No Grads'] = general_data_cluster_dropout_rate['# Total Cohort'] - general_data_cluster_dropout_rate['# Grads']
general_data_cluster_dropout_rate['% Dropout'] = general_data_cluster_dropout_rate['# Dropout'] / general_data_cluster_dropout_rate['# No Grads'] * 100
general_data_cluster_dropout_rate

,Graduation Year,Duration,Cluster,# Dropout,# Grads,# Total Cohort,# No Grads,% Dropout
0,2018,4,0,969.0,1169.0,5003,3834.0,25.273865
1,2018,4,1,3362.0,22878.0,31418,8540.0,39.367681
2,2018,4,2,788.0,32741.0,35675,2934.0,26.857532
3,2018,5,0,1687.0,2130.0,5557,3427.0,49.226729
4,2018,5,1,4870.0,22997.0,30233,7236.0,67.302377
...,...,...,...,...,...,...,...,...
58,2024,5,1,2732.0,23879.0,27408,3529.0,77.415698
59,2024,5,2,915.0,33632.0,35000,1368.0,66.885965
60,2024,6,0,1246.0,2145.0,3809,1664.0,74.879808
61,2024,6,1,2942.0,24626.0,28123,3497.0,84.129254


In [207]:
fig = make_subplots(rows=1, cols=len(general_data_cluster_dropout_rate['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in general_data_cluster_dropout_rate['Cluster'].unique():
    cluster_row = general_data_cluster_dropout_rate[general_data_cluster_dropout_rate['Cluster'] == cluster]
    cluster_index = general_data_cluster_dropout_rate['Cluster'].unique().tolist().index(cluster)
    for duration in cluster_row['Duration'].unique():
        duration_row = cluster_row[cluster_row['Duration'] == duration]
        duration_index = cluster_row['Duration'].unique().tolist().index(duration)
        fig.add_trace(go.Scatter(x=duration_row['Graduation Year'], y=duration_row['% Dropout'], name=f'{duration} Years', line=dict(color=colors[duration_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(title_text='Dropout %', range=[general_data_cluster_dropout_rate['% Dropout'].min(),general_data_cluster_dropout_rate['% Dropout'].max()])
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text = "Dropout Rate in Every Cluster per Duration")

for i in range(len(fig.data) - len(colors)):
    fig.data[i].showlegend = False
fig.show()

### Test Results

In [62]:
general_test_results

,School Name,Cluster,Year,Category,Regents Exam,Total Tested,Number Scoring 65 or Above,Regents Subject
0,A-TECH HIGH SCHOOL,1,2015,All Students,Algebra2/Trigonometry,7,0.0,Mathematics
1,A-TECH HIGH SCHOOL,1,2015,All Students,Common Core Algebra,41,5.0,Mathematics
2,A-TECH HIGH SCHOOL,1,2015,All Students,Common Core English,167,63.0,English
3,A-TECH HIGH SCHOOL,1,2015,All Students,Common Core Geometry,26,2.0,Mathematics
4,A-TECH HIGH SCHOOL,1,2015,All Students,English,109,43.0,English
...,...,...,...,...,...,...,...,...
27299,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,All Students,Living Environment,107,79.0,Science
27300,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,All Students,Physical Settings/Chemistry,81,34.0,Science
27301,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,All Students,Physical Settings/Earth Science,60,47.0,Science
27302,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,All Students,Spanish,4,NaN,Languages Other Than English


In [63]:
general_data_cluster_test_results = general_test_results.groupby(['Cluster', 'Regents Subject'])[['Total Tested', 'Number Scoring 65 or Above']].sum().reset_index()
general_data_cluster_test_results['% Passed'] = general_data_cluster_test_results['Number Scoring 65 or Above'] / general_data_cluster_test_results['Total Tested'] * 100
general_data_cluster_test_results

,Cluster,Regents Subject,Total Tested,Number Scoring 65 or Above,% Passed
0,0,English,22311,13531.0,60.647214
1,0,Languages Other Than English,1813,1633.0,90.071704
2,0,Mathematics,23496,9915.0,42.198672
3,0,Science,18986,8916.0,46.960919
4,0,Social Studies,35413,17031.0,48.092508
5,1,English,239104,160208.0,67.003480
6,1,Languages Other Than English,52521,45975.0,87.536414
7,1,Mathematics,474684,210021.0,44.244382
8,1,Science,354854,172414.0,48.587306
9,1,Social Studies,381790,220349.0,57.714712


In [228]:
fig = make_subplots(rows=1, cols=len(general_data_cluster_test_results['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in general_data_cluster_test_results['Cluster'].unique():
    cluster_row = general_data_cluster_test_results[general_data_cluster_test_results['Cluster'] ==  cluster]
    cluster_index = general_data_cluster_test_results['Cluster'].unique().tolist().index(cluster)
    for subject in cluster_row['Regents Subject'].unique():
        subject_row = cluster_row[cluster_row['Regents Subject'] == subject]
        subject_index = cluster_row['Regents Subject'].unique().tolist().index(subject)
        fig.add_trace(go.Bar(x=subject_row['Regents Subject'], y=subject_row['% Passed'], showlegend=False, marker=dict(color=colors2[subject_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, general_data_cluster_test_results['% Passed'].max()], title_text='% of Approved Students')

fig.update_layout(
    margin=dict(b=125),
    title_text='Rate of Students Who Approved Each Regents Subject in Every Cluster'
)
fig.show()

In [121]:
general_data_cluster_test_results_per_year = general_test_results.groupby(['Cluster', 'Regents Subject', 'Year'])[['Total Tested', 'Number Scoring 65 or Above']].sum().reset_index()
general_data_cluster_test_results_per_year['% Passed'] = general_data_cluster_test_results_per_year['Number Scoring 65 or Above'] / general_data_cluster_test_results_per_year['Total Tested'] * 100
general_data_cluster_test_results_per_year

,Cluster,Regents Subject,Year,Total Tested,Number Scoring 65 or Above,% Passed
0,0,English,2015,4127,2728.0,66.101284
1,0,English,2016,4145,2434.0,58.721351
2,0,English,2017,3290,2051.0,62.340426
3,0,English,2018,3559,2027.0,56.954201
4,0,English,2019,3425,2222.0,64.875912
...,...,...,...,...,...,...
100,2,Social Studies,2017,64868,51651.0,79.624776
101,2,Social Studies,2018,65731,53211.0,80.952671
102,2,Social Studies,2019,65333,53205.0,81.436640
103,2,Social Studies,2022,29303,24338.0,83.056342


In [229]:
fig = make_subplots(rows=1, cols=len(general_data_cluster_test_results_per_year['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in general_data_cluster_test_results_per_year['Cluster'].unique():
    cluster_row = general_data_cluster_test_results_per_year[general_data_cluster_test_results_per_year['Cluster'] ==  cluster]
    cluster_index = general_data_cluster_test_results_per_year['Cluster'].unique().tolist().index(cluster)
    for subject in cluster_row['Regents Subject'].unique():
        subject_row = cluster_row[cluster_row['Regents Subject'] == subject]
        subject_index = cluster_row['Regents Subject'].unique().tolist().index(subject)
        fig.add_trace(go.Scatter(x=subject_row['Year'], y=subject_row['% Passed'], name=subject, marker=dict(color=colors2[subject_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[general_data_cluster_test_results_per_year['% Passed'].min(), general_data_cluster_test_results_per_year['% Passed'].max()], title_text='% of Approved Students')
        fig.update_xaxes(title_text='Year')

for i in range(len(fig.data) - len(general_data_cluster_test_results_per_year['Regents Subject'].unique())):
    fig.data[i].showlegend=False

fig.update_layout(title_text='Rate of Students who Approved each Regents Subject in Every Cluster from 2015 to 2023')

fig.show()

## Impact of Ethnicity

In [67]:
all_ethnicity_graduates = filter_data(ethnicity_graduation, columns=['# Grads','# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout'], cluster=True)
all_ethnicity_graduates

,School Name,Category,Graduation Year,Duration,Cluster,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,# Total Cohort,Graduation Month
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Asian,2018,6,2,NaN,NaN,NaN,NaN,NaN,NaN,1,June
1,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Asian,2020,4,2,NaN,NaN,NaN,NaN,NaN,NaN,2,August
2,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Asian,2021,5,2,NaN,NaN,NaN,NaN,NaN,NaN,2,August
3,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Asian,2022,4,2,NaN,NaN,NaN,NaN,NaN,NaN,3,August
4,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Asian,2022,6,2,NaN,NaN,NaN,NaN,NaN,NaN,2,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46301,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",White,2023,5,2,NaN,NaN,NaN,NaN,NaN,NaN,1,August
46302,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",White,2023,6,2,6.0,6.0,0.0,0.0,0.0,0.0,6,June
46303,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",White,2024,4,2,NaN,NaN,NaN,NaN,NaN,NaN,1,August
46304,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",White,2024,5,2,NaN,NaN,NaN,NaN,NaN,NaN,3,August


In [68]:
ethnicity_data = filter_graduation_data(all_ethnicity_graduates, ['Graduation Year','Category','Duration'], cluster=True)
ethnicity_data

,Graduation Year,Category,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout
0,2018,Asian,4,0,331,97.0,40.0,57.0,0.0,118.0,60.0
1,2018,Asian,4,1,4021,2910.0,910.0,1873.0,127.0,564.0,299.0
2,2018,Asian,4,2,8618,7930.0,5523.0,2294.0,113.0,456.0,79.0
3,2018,Asian,5,0,365,123.0,53.0,68.0,2.0,87.0,88.0
4,2018,Asian,5,1,3886,2959.0,936.0,1919.0,104.0,198.0,398.0
...,...,...,...,...,...,...,...,...,...,...,...
373,2024,White,5,1,2420,1852.0,580.0,1266.0,6.0,20.0,144.0
374,2024,White,5,2,6867,6422.0,3426.0,2979.0,17.0,37.0,102.0
375,2024,White,6,0,185,76.0,0.0,76.0,0.0,4.0,34.0
376,2024,White,6,1,2446,1897.0,763.0,1125.0,9.0,7.0,138.0


In [69]:
ethnicity_data['Category Code'] = ethnicity_data['Category'].astype('category').cat.codes
ethnicity_data

,Graduation Year,Category,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,Category Code
0,2018,Asian,4,0,331,97.0,40.0,57.0,0.0,118.0,60.0,0
1,2018,Asian,4,1,4021,2910.0,910.0,1873.0,127.0,564.0,299.0,0
2,2018,Asian,4,2,8618,7930.0,5523.0,2294.0,113.0,456.0,79.0,0
3,2018,Asian,5,0,365,123.0,53.0,68.0,2.0,87.0,88.0,0
4,2018,Asian,5,1,3886,2959.0,936.0,1919.0,104.0,198.0,398.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
373,2024,White,5,1,2420,1852.0,580.0,1266.0,6.0,20.0,144.0,5
374,2024,White,5,2,6867,6422.0,3426.0,2979.0,17.0,37.0,102.0,5
375,2024,White,6,0,185,76.0,0.0,76.0,0.0,4.0,34.0,5
376,2024,White,6,1,2446,1897.0,763.0,1125.0,9.0,7.0,138.0,5


### Correlation Matrix

In [140]:
correlation_matrix(ethnicity_data.drop('Category', axis=1))

### Population

In [71]:
def filter_population(df):
    df_population = df.groupby(['Category', 'Cluster'])[['# Total Cohort', '# Grads']].sum().reset_index()
    df_population['General Cohort %'] = df_population['# Total Cohort'] / df_population['Cluster'].map(general_data_cluster_population.set_index('Cluster')['# Total Cohort']) * 100
    return df_population

In [72]:
ethnicity_data_population = filter_population(ethnicity_data)
ethnicity_data_population

,Category,Cluster,# Total Cohort,# Grads,General Cohort %
0,Asian,0,6569,2559.0,6.398356
1,Asian,1,81475,63906.0,13.157873
2,Asian,2,178022,166972.0,24.058588
3,Black,0,39987,17032.0,38.948250
4,Black,1,174201,138607.0,28.132737
5,Black,2,157631,141146.0,21.302868
6,Hispanic,0,48737,18336.0,47.470950
7,Hispanic,1,297972,229270.0,48.121238
8,Hispanic,2,236243,215641.0,31.926801
9,Multi-Racial,0,582,65.0,0.566881


In [209]:
fig = make_subplots(rows=len(ethnicity_data_population['Cluster'].unique()), cols=1, subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ethnicity_data_population['Cluster'].unique():
    cluster_row = ethnicity_data_population[ethnicity_data_population['Cluster'] == cluster]
    cluster_index = ethnicity_data_population['Cluster'].unique().tolist().index(cluster)
    for ethnicity in cluster_row['Category'].unique():
        ethnicity_row = cluster_row[cluster_row['Category'] == ethnicity]
        ethnicity_index = cluster_row['Category'].unique().tolist().index(ethnicity)
        fig.add_trace(go.Bar(x=ethnicity_row['Category'], y=ethnicity_row['General Cohort %'], marker=dict(color=colors2[ethnicity_index]), showlegend=False), row=cluster_index+1, col=1)
        fig.update_yaxes(range=[0,ethnicity_data_population['General Cohort %'].max()], title_text='Distribution %')

fig.update_layout(
    width=1200,
    height=900,
    title_text = "Population Distribution in Every Cluster per Ethnicity",
)
fig.show()

### Graduation Rate

In [74]:
def filter_graduation_rate(df):
    df_graduation_rate = df.groupby(['Category', 'Graduation Year', 'Cluster'])[['# Total Cohort', '# Grads']].sum().reset_index()
    df_graduation_rate['Grad %'] = df_graduation_rate['# Grads'] / df_graduation_rate['# Total Cohort'] * 100
    return df_graduation_rate

In [75]:
ethnicity_data_graduation_rate = filter_graduation_rate(ethnicity_data)
ethnicity_data_graduation_rate

,Category,Graduation Year,Cluster,# Total Cohort,# Grads,Grad %
0,Asian,2018,0,1127,395.0,35.048802
1,Asian,2018,1,11934,8875.0,74.367354
2,Asian,2018,2,24496,22656.0,92.488570
3,Asian,2019,0,1097,414.0,37.739289
4,Asian,2019,1,11911,8982.0,75.409286
...,...,...,...,...,...,...
121,White,2023,1,7469,5680.0,76.047664
122,White,2023,2,21191,19757.0,93.232976
123,White,2024,0,564,203.0,35.992908
124,White,2024,1,7252,5510.0,75.979040


In [210]:
fig = make_subplots(rows=1, cols=len(ethnicity_data_graduation_rate['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ethnicity_data_graduation_rate['Cluster'].unique():
    cluster_row = ethnicity_data_graduation_rate[ethnicity_data_graduation_rate['Cluster'] == cluster]
    cluster_index = ethnicity_data_graduation_rate['Cluster'].unique().tolist().index(cluster)
    for ethnicity in cluster_row['Category'].unique():
        ethnicity_row = cluster_row[cluster_row['Category'] == ethnicity]
        ethnicity_index = cluster_row['Category'].unique().tolist().index(ethnicity)
        fig.add_trace(go.Scatter(x=ethnicity_row['Graduation Year'], y=ethnicity_row['Grad %'], name=ethnicity, line=dict(color=colors2[ethnicity_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0,ethnicity_data_graduation_rate['Grad %'].max()], title_text='Graduation %')
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text='Graduation Rate from 2018 to 2024 in Every Cluster per Ethnicity')

for i in range(len(fig.data) - len(ethnicity_data_graduation_rate['Category'].unique())):
    fig.data[i].showlegend = False
fig.show()

### Diploma Type

In [77]:
def filter_diploma_type(df):
    df_diploma_type = df.groupby(['Category','Cluster'])[['# Advanced Regents', '# Regents without Advanced', '# Local', '# Grads']].sum().reset_index()
    df_diploma_type['Advanced Regents %'] = df_diploma_type['# Advanced Regents'] / df_diploma_type['# Grads'] * 100
    df_diploma_type['Regents without Advanced %'] = df_diploma_type['# Regents without Advanced'] / df_diploma_type['# Grads'] * 100
    df_diploma_type['Local %'] = df_diploma_type['# Local'] / df_diploma_type['# Grads'] * 100
    return df_diploma_type

In [78]:
ethnicity_data_diploma_type = filter_diploma_type(ethnicity_data)
ethnicity_data_diploma_type

,Category,Cluster,# Advanced Regents,# Regents without Advanced,# Local,# Grads,Advanced Regents %,Regents without Advanced %,Local %
0,Asian,0,803.0,1727.0,29.0,2559.0,31.379445,67.487300,1.133255
1,Asian,1,24660.0,37896.0,1352.0,63906.0,38.587926,59.299596,2.115607
2,Asian,2,119629.0,46187.0,1207.0,166972.0,71.646144,27.661524,0.722876
3,Black,0,134.0,14540.0,2364.0,17032.0,0.786754,85.368718,13.879756
4,Black,1,14295.0,113722.0,10617.0,138607.0,10.313332,82.046361,7.659786
5,Black,2,33696.0,101313.0,6153.0,141146.0,23.873153,71.778867,4.359316
6,Hispanic,0,299.0,15786.0,2252.0,18336.0,1.630672,86.092932,12.281850
7,Hispanic,1,33557.0,179668.0,16071.0,229270.0,14.636455,78.365246,7.009639
8,Hispanic,2,76235.0,132036.0,7423.0,215641.0,35.352739,61.229544,3.442295
9,Multi-Racial,0,0.0,65.0,0.0,65.0,0.000000,100.000000,0.000000


In [211]:
fig = make_subplots(rows=len(ethnicity_data_diploma_type['Cluster'].unique()), cols=1, subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ethnicity_data_diploma_type['Cluster'].unique():
    cluster_row = ethnicity_data_diploma_type[ethnicity_data_diploma_type['Cluster'] == cluster]
    cluster_index = ethnicity_data_diploma_type['Cluster'].unique().tolist().index(cluster)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Advanced Regents %'], name='Advanced Regents Diploma', marker=dict(color=colors[0])), row=cluster_index+1, col=1)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Regents without Advanced %'], name='Regular Diploma', marker=dict(color=colors[1])), row=cluster_index+1, col=1)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Local %'], name='Local Diploma', marker=dict(color=colors[2])), row=cluster_index+1, col=1)
    fig.update_yaxes(range=[0, max(ethnicity_data_diploma_type[['Advanced Regents %', 'Regents without Advanced %', 'Local %']].max(axis=1))])

fig.update_layout(
    width=1200,
    height=900,
    title_text = "Distribution of Graduates in Every Cluster per Ethnicity by Type of Diploma",
)

for i in range(len(fig.data) - 3):
    fig.data[i].showlegend = False
fig.show()

### Dropouts

In [80]:
def filter_dropout(df):
    df_dropout = df.groupby(['Category', 'Cluster'])[['# Dropout', '# Total Cohort', '# Grads']].sum().reset_index()
    df_dropout['# No Grads'] = df_dropout['# Total Cohort'] - df_dropout['# Grads']
    df_dropout['Dropout %'] = df_dropout['# Dropout'] / df_dropout['# No Grads'] * 100
    return df_dropout

In [81]:
ethnicity_data_dropout = filter_dropout(ethnicity_data)
ethnicity_data_dropout

,Category,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Asian,0,1180.0,6569,2559.0,4010.0,29.426434
1,Asian,1,6991.0,81475,63906.0,17569.0,39.791679
2,Asian,2,2126.0,178022,166972.0,11050.0,19.239819
3,Black,0,8179.0,39987,17032.0,22955.0,35.630582
4,Black,1,17781.0,174201,138607.0,35594.0,49.955049
5,Black,2,5181.0,157631,141146.0,16485.0,31.428571
6,Hispanic,0,14081.0,48737,18336.0,30401.0,46.317555
7,Hispanic,1,41354.0,297972,229270.0,68702.0,60.193299
8,Hispanic,2,9029.0,236243,215641.0,20602.0,43.825842
9,Multi-Racial,0,4.0,582,65.0,517.0,0.773694


In [212]:
fig = make_subplots(rows=len(ethnicity_data_dropout['Cluster'].unique()), cols=1, subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ethnicity_data_dropout['Cluster'].unique():
    cluster_row = ethnicity_data_dropout[ethnicity_data_dropout['Cluster'] == cluster]
    cluster_index = ethnicity_data_dropout['Cluster'].unique().tolist().index(cluster)
    for ethnicity in cluster_row['Category'].unique():
        ethnicity_row = cluster_row[cluster_row['Category'] == ethnicity]
        ethnicity_index = cluster_row['Category'].unique().tolist().index(ethnicity)
        fig.add_trace(go.Bar(x=ethnicity_row['Category'], y=ethnicity_row['Dropout %'], name=ethnicity, marker=dict(color=colors2[ethnicity_index]), showlegend=False), row=cluster_index + 1, col=1)
        fig.update_yaxes(range=[0, ethnicity_data_dropout['Dropout %'].max()], title_text='Dropout %')
        fig.update_xaxes(title_text='Ethnicity')

fig.update_layout(
    width=1200,
    height=900,
    title_text = "Dropout Rate in Every Cluster per Ethnicity",
)

fig.show()

In [83]:
def filter_dropout_per_year(df):
    df_dropout_per_year = df.groupby(['Category', 'Graduation Year', 'Cluster'])[['# Dropout', '# Total Cohort', '# Grads']].sum().reset_index()
    df_dropout_per_year['# No Grads'] = df_dropout_per_year['# Total Cohort'] - df_dropout_per_year['# Grads']
    df_dropout_per_year['Dropout %'] = df_dropout_per_year['# Dropout'] / df_dropout_per_year['# No Grads'] * 100
    return df_dropout_per_year

In [84]:
ethnicity_data_dropout_per_year = filter_dropout_per_year(ethnicity_data)
ethnicity_data_dropout_per_year

,Category,Graduation Year,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Asian,2018,0,264.0,1127,395.0,732.0,36.065574
1,Asian,2018,1,1178.0,11934,8875.0,3059.0,38.509317
2,Asian,2018,2,371.0,24496,22656.0,1840.0,20.163043
3,Asian,2019,0,232.0,1097,414.0,683.0,33.967789
4,Asian,2019,1,1247.0,11911,8982.0,2929.0,42.574257
...,...,...,...,...,...,...,...,...
121,White,2023,1,405.0,7469,5680.0,1789.0,22.638345
122,White,2023,2,304.0,21191,19757.0,1434.0,21.199442
123,White,2024,0,62.0,564,203.0,361.0,17.174515
124,White,2024,1,358.0,7252,5510.0,1742.0,20.551091


In [213]:
fig = make_subplots(rows=1, cols=len(ethnicity_data_dropout_per_year['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ethnicity_data_dropout_per_year['Cluster'].unique():
    cluster_row = ethnicity_data_dropout_per_year[ethnicity_data_dropout_per_year['Cluster'] == cluster]
    cluster_index = ethnicity_data_dropout_per_year['Cluster'].unique().tolist().index(cluster)
    for ethnicity in cluster_row['Category'].unique():
        ethnicity_row = cluster_row[cluster_row['Category'] == ethnicity]
        ethnicity_index = cluster_row['Category'].unique().tolist().index(ethnicity)
        fig.add_trace(go.Scatter(x=ethnicity_row['Graduation Year'], y=ethnicity_row['Dropout %'], name=ethnicity, line=dict(color=colors2[ethnicity_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, ethnicity_data_dropout_per_year['Dropout %'].max()], title_text = 'Dropout %')
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text='Dropout Rates from 2018 to 2024 in Every Cluster per Ethnicity')

for i in range(len(fig.data) - len(ethnicity_data_dropout_per_year['Category'].unique())):
    fig.data[i].showlegend = False

fig.show()

In [86]:
def filter_dropout_per_duration(df):
    df_dropout = filter_dropout(df)
    df_dropout_per_duration = df.groupby(['Category', 'Duration', 'Cluster'])['# Dropout'].sum().reset_index()
    df_dropout_per_duration = pd.merge(df_dropout_per_duration, df_dropout.drop(columns=['# Total Cohort', '# Grads', '# No Grads', 'Dropout %']), on=['Category','Cluster'], how='outer')
    df_dropout_per_duration['Dropout %'] = df_dropout_per_duration['# Dropout_x'] / df_dropout_per_duration['# Dropout_y'] * 100
    return df_dropout_per_duration

In [87]:
ethnicity_data_dropout_per_duration = filter_dropout_per_duration(ethnicity_data)
ethnicity_data_dropout_per_duration

,Category,Duration,Cluster,# Dropout_x,# Dropout_y,Dropout %
0,Asian,4,0,269.0,1180.0,22.796610
1,Asian,5,0,392.0,1180.0,33.220339
2,Asian,6,0,519.0,1180.0,43.983051
3,Asian,4,1,1764.0,6991.0,25.232442
4,Asian,5,1,2453.0,6991.0,35.087970
5,Asian,6,1,2774.0,6991.0,39.679588
6,Asian,4,2,425.0,2126.0,19.990593
7,Asian,5,2,841.0,2126.0,39.557855
8,Asian,6,2,860.0,2126.0,40.451552
9,Black,4,0,1445.0,8179.0,17.667196


In [214]:
fig = make_subplots(rows=len(ethnicity_data_dropout_per_duration['Cluster'].unique()), cols=1, subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ethnicity_data_dropout_per_duration['Cluster'].unique():
    cluster_row = ethnicity_data_dropout_per_duration[ethnicity_data_dropout_per_duration['Cluster'] == cluster]
    cluster_index = ethnicity_data_dropout_per_duration['Cluster'].unique().tolist().index(cluster)
    for duration in cluster_row['Duration'].unique():
        duration_row = cluster_row[cluster_row['Duration'] == duration]
        duration_index = cluster_row['Duration'].unique().tolist().index(duration)
        fig.add_trace(go.Bar(x=duration_row['Category'], y=duration_row['Dropout %'], marker=dict(color=colors[duration_index]), name=f'{duration} Years'), row=cluster_index+1, col=1)
        fig.update_yaxes(range=[0, ethnicity_data_dropout_per_duration['Dropout %'].max()], title_text='Distribution %')
        fig.update_xaxes(title_text='Ethnicity')

for i in range(len(fig.data) - len(ethnicity_data_dropout_per_duration['Duration'].unique())):
    fig.data[i].showlegend = False

fig.update_layout(
    width=1200,
    height=900,
    title_text = "Distribution of Dropouts in Every Cluster per Ethnicity Based on Duration in School",
)

fig.show()

### Test Results

In [190]:
ethnicity_test_results

,School Name,Cluster,Year,Category,Regents Exam,Total Tested,Number Scoring 65 or Above,Regents Subject
0,A-TECH HIGH SCHOOL,1,2015,Black,Algebra2/Trigonometry,1,NaN,Mathematics
1,A-TECH HIGH SCHOOL,1,2015,Hispanic,Algebra2/Trigonometry,6,NaN,Mathematics
2,A-TECH HIGH SCHOOL,1,2015,Multiple Race Categories Not Represented,Common Core Algebra,1,NaN,Mathematics
3,A-TECH HIGH SCHOOL,1,2015,White,Common Core Algebra,1,NaN,Mathematics
4,A-TECH HIGH SCHOOL,1,2015,Hispanic,Common Core Algebra,15,2.0,Mathematics
...,...,...,...,...,...,...,...,...
108069,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,White,US History and Government,2,NaN,Social Studies
108070,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,Multiple Race Categories Not Represented,US History and Government,6,NaN,Social Studies
108071,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,Hispanic,US History and Government,12,9.0,Social Studies
108072,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,Black,US History and Government,21,21.0,Social Studies


In [164]:
ethnicity_data_test_results = ethnicity_test_results.groupby(['Category', 'Cluster', 'Regents Subject'])[['Total Tested', 'Number Scoring 65 or Above']].sum().reset_index()
ethnicity_data_test_results['% Passed'] = ethnicity_data_test_results['Number Scoring 65 or Above'] / ethnicity_data_test_results['Total Tested'] * 100
ethnicity_data_test_results

,Category,Cluster,Regents Subject,Total Tested,Number Scoring 65 or Above,% Passed
0,Asian,0,English,1671,947.0,56.672651
1,Asian,0,Languages Other Than English,477,277.0,58.071279
2,Asian,0,Mathematics,2744,1773.0,64.613703
3,Asian,0,Science,1927,1231.0,63.881681
4,Asian,0,Social Studies,2421,1490.0,61.544816
...,...,...,...,...,...,...
70,White,2,English,44481,39091.0,87.882467
71,White,2,Languages Other Than English,21585,18601.0,86.175585
72,White,2,Mathematics,110443,77941.0,70.571245
73,White,2,Science,95404,69163.0,72.494864


In [216]:
fig = make_subplots(rows=2, cols=len(ethnicity_data_test_results['Category'].unique())//2+1, subplot_titles=(ethnicity_data_test_results['Category'].unique()))

for ethnicity in ethnicity_data_test_results['Category'].unique():
    ethnicity_row = ethnicity_data_test_results[ethnicity_data_test_results['Category'] == ethnicity]
    ethnicity_index = ethnicity_data_test_results['Category'].unique().tolist().index(ethnicity)
    if (ethnicity_index+1) % 3 == 0:
        col = 3
    else:
        col = (ethnicity_index+1) % 3
    for cluster in ethnicity_row['Cluster'].unique():
        cluster_row = ethnicity_row[ethnicity_row['Cluster'] == cluster]
        cluster_index = ethnicity_data_test_results['Cluster'].unique().tolist().index(cluster)
        fig.add_trace(go.Bar(x=cluster_row['Regents Subject'], y=cluster_row['% Passed'], name=f'Cluster {cluster}', marker=dict(color=colors[cluster_index])), row=math.ceil((ethnicity_index+1)/3), col=col)
        fig.update_yaxes(range=[0,ethnicity_data_test_results['% Passed'].max()], title_text='Approved %')

fig.update_layout(
    height=800,
    width=1200,
    title_text='Rate of Students Who Aproved each Regents Subject in Every Cluster per Ethnicity',
    margin=dict(b=125)
)

for i in range(len(fig.data) - len(ethnicity_data_test_results['Cluster'].unique())):
    fig.data[i].showlegend=False
fig.show()

## Impact of English Language Learning

In [89]:
all_ell_graduates = filter_data(ell_graduation, columns=['# Grads','# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout'], cluster=True)
all_ell_graduates.loc[all_ell_graduates['Category'] == 'Current ELL', 'Category'] = 'Ever ELL'
all_ell_graduates

,School Name,Category,Graduation Year,Duration,Cluster,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,# Total Cohort,Graduation Month
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Ever ELL,2018,4,2,NaN,NaN,NaN,NaN,NaN,NaN,1,August
1,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Ever ELL,2018,5,2,NaN,NaN,NaN,NaN,NaN,NaN,1,August
2,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Ever ELL,2018,6,2,NaN,NaN,NaN,NaN,NaN,NaN,2,June
3,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Ever ELL,2019,5,2,NaN,NaN,NaN,NaN,NaN,NaN,1,August
4,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Ever ELL,2019,6,2,NaN,NaN,NaN,NaN,NaN,NaN,1,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27376,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Never ELL,2023,5,2,59.0,48.0,11.0,0.0,0.0,1.0,60,August
27377,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Never ELL,2023,6,2,52.0,43.0,9.0,0.0,2.0,2.0,56,June
27378,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Never ELL,2024,4,2,54.0,40.0,14.0,0.0,0.0,0.0,54,August
27379,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Never ELL,2024,5,2,NaN,NaN,NaN,NaN,NaN,NaN,73,August


In [91]:
ell_data = filter_graduation_data(all_ell_graduates, ['Graduation Year','Category','Duration'], cluster=True)
ell_data['Category Code'] = ell_data['Category'].astype('category').cat.codes
ell_data

,Graduation Year,Category,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,Category Code
0,2018,Ever ELL,4,0,1590,320.0,40.0,244.0,36.0,613.0,345.0,0
1,2018,Ever ELL,4,1,12047,7075.0,893.0,5200.0,982.0,2429.0,1396.0,0
2,2018,Ever ELL,4,2,7453,4039.0,1678.0,2108.0,253.0,569.0,160.0,0
3,2018,Ever ELL,5,0,1567,430.0,48.0,283.0,99.0,351.0,478.0,0
4,2018,Ever ELL,5,1,10483,6141.0,758.0,4460.0,923.0,802.0,1830.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2024,Never ELL,5,1,21834,17478.0,3630.0,13777.0,73.0,197.0,1341.0,1
122,2024,Never ELL,5,2,33038,25256.0,11808.0,13393.0,55.0,141.0,508.0,1
123,2024,Never ELL,6,0,2781,1429.0,20.0,1344.0,65.0,127.0,753.0,1
124,2024,Never ELL,6,1,21870,17839.0,5486.0,12279.0,74.0,66.0,1520.0,1


### Correlation Matrix

In [141]:
correlation_matrix(ell_data.drop('Category', axis=1))

### Population

In [93]:
ell_data_population = filter_population(ell_data)
ell_data_population

,Category,Cluster,# Total Cohort,# Grads,General Cohort %
0,Ever ELL,0,33724,11197.0,32.847945
1,Ever ELL,1,240617,160225.0,38.858644
2,Ever ELL,2,174581,117858.0,23.593557
3,Never ELL,0,68943,28338.0,67.152055
4,Never ELL,1,378594,308864.0,61.141356
5,Never ELL,2,565371,507350.0,76.406443


In [217]:
fig = make_subplots(rows=1, cols=len(ell_data_population['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_population['Cluster'].unique():
    cluster_row = ell_data_population[ell_data_population['Cluster'] == cluster]
    cluster_index = ell_data_population['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Category'], y=category_row['General Cohort %'], name=category, marker=dict(color=colors[category_index]), showlegend=False), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, ell_data_population['General Cohort %'].max()], title_text='Distribution %')
        fig.update_xaxes(title_text='Category')

fig.update_layout(title_text='Distribution of Population in Every Cluster per English Language Learning (ELL) Status')
fig.show()

### Graduation Rate

In [95]:
ell_data_graduation_rate = filter_graduation_rate(ell_data)
ell_data_graduation_rate

,Category,Graduation Year,Cluster,# Total Cohort,# Grads,Grad %
0,Ever ELL,2018,0,4663,1199.0,25.713060
1,Ever ELL,2018,1,32152,18683.0,58.108360
2,Ever ELL,2018,2,18460,10732.0,58.136511
3,Ever ELL,2019,0,5430,1473.0,27.127072
4,Ever ELL,2019,1,37957,24182.0,63.708934
5,Ever ELL,2019,2,28330,18339.0,64.733498
6,Ever ELL,2020,0,5762,1873.0,32.506074
7,Ever ELL,2020,1,39501,26591.0,67.317283
8,Ever ELL,2020,2,30642,20702.0,67.560864
9,Ever ELL,2021,0,5422,2097.0,38.675765


In [218]:
fig = make_subplots(rows=1, cols=len(ell_data_graduation_rate['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_graduation_rate['Cluster'].unique():
    cluster_row = ell_data_graduation_rate[ell_data_graduation_rate['Cluster'] == cluster]
    cluster_index = ell_data_graduation_rate['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Scatter(x=category_row['Graduation Year'], y=category_row['Grad %'], name=category, marker=dict(color=colors[category_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[ell_data_graduation_rate['Grad %'].min(), ell_data_graduation_rate['Grad %'].max()], title_text='Graduation %')
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text='Graduation Rate from 2018 to 2024 in Every Cluster per English Language Learning (ELL) Status')

for i in range(len(fig.data) - len(ell_data_graduation_rate['Category'].unique())):
    fig.data[i].showlegend = False

fig.show()

### Diploma Type

In [97]:
ell_data_diploma_type = filter_diploma_type(ell_data)
ell_data_diploma_type

,Category,Cluster,# Advanced Regents,# Regents without Advanced,# Local,# Grads,Advanced Regents %,Regents without Advanced %,Local %
0,Ever ELL,0,994.0,9198.0,1005.0,11197.0,8.877378,82.147004,8.975618
1,Ever ELL,1,30075.0,119155.0,11001.0,160225.0,18.770479,74.367296,6.865970
2,Ever ELL,2,57529.0,56967.0,3391.0,117858.0,48.812130,48.335285,2.877191
3,Never ELL,0,273.0,24315.0,3751.0,28338.0,0.963371,85.803515,13.236643
4,Never ELL,1,55814.0,234809.0,18289.0,308864.0,18.070737,76.023428,5.921376
5,Never ELL,2,237503.0,257614.0,12309.0,507350.0,46.812457,50.776387,2.426136


In [220]:
fig = make_subplots(rows=1, cols=len(ell_data_diploma_type['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_diploma_type['Cluster'].unique():
    cluster_row = ell_data_diploma_type[ell_data_diploma_type['Cluster'] == cluster]
    cluster_index = ell_data_population['Cluster'].unique().tolist().index(cluster)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Advanced Regents %'], name='Advanced Regents Diploma', marker=dict(color=colors[0])), row=1, col=cluster_index+1)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Regents without Advanced %'], name='Regular Regents Diploma', marker=dict(color=colors[1])), row=1, col=cluster_index+1)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Local %'], name='Local Diploma', marker=dict(color=colors[2])), row=1, col=cluster_index+1)
    fig.update_yaxes(range=[0, max(ell_data_diploma_type[['Advanced Regents %', 'Regents without Advanced %', 'Local %']].max(axis=1))], title_text='Distribution %')

for i in range(len(fig.data) - 3):
    fig.data[i].showlegend = False

fig.update_layout(title_text='Distribution of Graduates in Every Cluster per English Language Learning (ELL) Status by Diploma Type')
fig.show()

### Dropouts

In [99]:
ell_data_dropout = filter_dropout(ell_data)
ell_data_dropout

,Category,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Ever ELL,0,8154.0,33724,11197.0,22527.0,36.196564
1,Ever ELL,1,31409.0,240617,160225.0,80392.0,39.069808
2,Ever ELL,2,5051.0,174581,117858.0,56723.0,8.904677
3,Never ELL,0,16166.0,68943,28338.0,40605.0,39.812831
4,Never ELL,1,38686.0,378594,308864.0,69730.0,55.479707
5,Never ELL,2,12985.0,565371,507350.0,58021.0,22.379828


In [100]:
fig = make_subplots(rows=1, cols=len(ell_data_dropout['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_dropout['Cluster'].unique():
    cluster_row = ell_data_dropout[ell_data_dropout['Cluster'] == cluster]
    cluster_index = ell_data_dropout['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Category'], y=category_row['Dropout %'], marker=dict(color=colors[category_index]), showlegend=False), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0,ell_data_dropout['Dropout %'].max()], title_text='Dropoout %')

fig.update_layout(title_text='Dropout Rate per ELL Level in Every Cluster')
fig.show()

In [101]:
ell_data_dropout_per_year = filter_dropout_per_year(ell_data)
ell_data_dropout_per_year

,Category,Graduation Year,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Ever ELL,2018,0,1271.0,4663,1199.0,3464.0,36.691686
1,Ever ELL,2018,1,5103.0,32152,18683.0,13469.0,37.887000
2,Ever ELL,2018,2,655.0,18460,10732.0,7728.0,8.475673
3,Ever ELL,2019,0,1496.0,5430,1473.0,3957.0,37.806419
4,Ever ELL,2019,1,5892.0,37957,24182.0,13775.0,42.773140
5,Ever ELL,2019,2,878.0,28330,18339.0,9991.0,8.787909
6,Ever ELL,2020,0,1295.0,5762,1873.0,3889.0,33.299049
7,Ever ELL,2020,1,5377.0,39501,26591.0,12910.0,41.649884
8,Ever ELL,2020,2,858.0,30642,20702.0,9940.0,8.631791
9,Ever ELL,2021,0,1037.0,5422,2097.0,3325.0,31.187970


In [119]:
fig = make_subplots(rows=1, cols=len(ell_data_dropout_per_year['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_dropout_per_year['Cluster'].unique():
    cluster_row = ell_data_dropout_per_year[ell_data_dropout_per_year['Cluster'] == cluster]
    cluster_index = ell_data_dropout_per_year['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Scatter(x=category_row['Graduation Year'], y=category_row['Dropout %'], line=dict(color=colors[category_index]), name=category), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[ell_data_dropout_per_year['Dropout %'].min(),ell_data_dropout_per_year['Dropout %'].max()], title_text='Dropoout %')
        fig.update_xaxes(title_text='Year')

for i in range(len(fig.data) - len(ell_data_dropout_per_year['Category'].unique())):
    fig.data[i].showlegend=False

fig.update_layout(title_text='Dropout Rate from 2018 to 2024 per ELL Level in Every Cluster')
fig.show()

In [103]:
ell_data_dropout_per_duration = filter_dropout_per_duration(ell_data)
ell_data_dropout_per_duration

,Category,Duration,Cluster,# Dropout_x,# Dropout_y,Dropout %
0,Ever ELL,4,0,1912.0,8154.0,23.448614
1,Ever ELL,5,0,2930.0,8154.0,35.933284
2,Ever ELL,6,0,3312.0,8154.0,40.618102
3,Ever ELL,4,1,7986.0,31409.0,25.425833
4,Ever ELL,5,1,11269.0,31409.0,35.878251
5,Ever ELL,6,1,12154.0,31409.0,38.695915
6,Ever ELL,4,2,1231.0,5051.0,24.371412
7,Ever ELL,5,2,1918.0,5051.0,37.972679
8,Ever ELL,6,2,1902.0,5051.0,37.655910
9,Never ELL,4,0,2855.0,16166.0,17.660522


In [104]:
fig = make_subplots(rows=1, cols=len(ell_data_dropout_per_duration['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_dropout_per_duration['Cluster'].unique():
    cluster_row = ell_data_dropout_per_duration[ell_data_dropout_per_duration['Cluster'] == cluster]
    cluster_index = ell_data_dropout_per_duration['Cluster'].unique().tolist().index(cluster)
    for duration in cluster_row['Duration'].unique():
        duration_row = cluster_row[cluster_row['Duration'] == duration]
        duration_index = cluster_row['Duration'].unique().tolist().index(duration)
        fig.add_trace(go.Bar(x=duration_row['Category'], y=duration_row['Dropout %'], name=f'{duration} Years', marker=dict(color=colors[duration_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, ell_data_dropout_per_duration['Dropout %'].max()], title_text='Distribution %')

for i in range(len(fig.data) - len(ell_data_dropout_per_duration['Duration'].unique())):
    fig.data[i].showlegend=False

fig.update_layout(title_text='Distribution of Dropouts per English Language Learning (ELL) Status Based on Duration in School')
fig.show()

### Test Results

In [105]:
ell_test_results.loc[ell_test_results['Category'] == 'Former ELL', 'Category'] = 'English Proficient'

In [106]:
ell_test_results

,School Name,Cluster,Year,Category,Regents Exam,Total Tested,Number Scoring 65 or Above,Regents Subject
0,A-TECH HIGH SCHOOL,1,2015,English Proficient,Algebra2/Trigonometry,1,NaN,Mathematics
1,A-TECH HIGH SCHOOL,1,2015,English Proficient,Algebra2/Trigonometry,6,NaN,Mathematics
2,A-TECH HIGH SCHOOL,1,2015,ELL,Common Core Algebra,4,NaN,Mathematics
3,A-TECH HIGH SCHOOL,1,2015,English Proficient,Common Core Algebra,5,NaN,Mathematics
4,A-TECH HIGH SCHOOL,1,2015,English Proficient,Common Core Algebra,32,5.0,Mathematics
...,...,...,...,...,...,...,...,...
71686,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,English Proficient,Spanish,1,NaN,Languages Other Than English
71687,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,English Proficient,Spanish,3,NaN,Languages Other Than English
71688,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,ELL,US History and Government,2,NaN,Social Studies
71689,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,English Proficient,US History and Government,17,NaN,Social Studies


In [107]:
ell_data_test_results = ell_test_results.groupby(['Category', 'Cluster', 'Regents Subject'])[['Number Scoring 65 or Above', 'Total Tested']].sum().reset_index()
ell_data_test_results['% Passed'] = ell_data_test_results['Number Scoring 65 or Above'] / ell_data_test_results['Total Tested'] * 100
ell_data_test_results

,Category,Cluster,Regents Subject,Number Scoring 65 or Above,Total Tested,% Passed
0,ELL,0,English,2126.0,4735,44.899683
1,ELL,0,Languages Other Than English,895.0,1305,68.582375
2,ELL,0,Mathematics,3614.0,7999,45.180648
3,ELL,0,Science,2609.0,6063,43.031503
4,ELL,0,Social Studies,3131.0,6676,46.899341
5,ELL,1,English,19258.0,53147,36.235347
6,ELL,1,Languages Other Than English,8970.0,11449,78.347454
7,ELL,1,Mathematics,32324.0,91777,35.220153
8,ELL,1,Science,20090.0,67004,29.983285
9,ELL,1,Social Studies,26366.0,70237,37.538619


In [113]:
fig = make_subplots(rows=1, cols=len(ell_data_test_results['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in ell_data_test_results['Cluster'].unique():
    cluster_row = ell_data_test_results[ell_data_test_results['Cluster'] == cluster]
    cluster_index = ell_data_test_results['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Regents Subject'], y=category_row['% Passed'], marker=dict(color=colors[category_index]), name=category), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, ell_data_test_results['% Passed'].max()], title_text='% Approved')

fig.update_layout(
    margin=dict(b=125),
    title_text='Rate of Students who Aproved Every Regents Subject in Every Cluster by English Language Learning (ELL) Status'
)

for i in range(len(fig.data) - len(ell_data_test_results['Category'].unique())):
    fig.data[i].showlegend=False
fig.show()

## Impact of Students With Disadvantages (SWD)

In [143]:
all_swd_graduates = filter_data(swd_graduation, columns=['# Grads','# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout'], cluster=True)
all_swd_graduates

,School Name,Category,Graduation Year,Duration,Cluster,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,# Total Cohort,Graduation Month
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Not SWD,2018,4,2,23.0,7.0,16.0,0.0,1.0,2.0,26,August
1,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Not SWD,2018,5,2,28.0,4.0,24.0,0.0,0.0,1.0,29,August
2,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Not SWD,2018,6,2,42.0,8.0,33.0,1.0,0.0,2.0,44,June
3,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Not SWD,2019,4,2,29.0,6.0,21.0,2.0,0.0,1.0,30,August
4,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Not SWD,2019,5,2,24.0,7.0,17.0,0.0,0.0,2.0,26,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19538,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",SWD,2023,5,2,9.0,6.0,3.0,0.0,0.0,0.0,9,August
19539,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",SWD,2023,6,2,7.0,4.0,3.0,0.0,1.0,1.0,9,June
19540,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",SWD,2024,4,2,NaN,NaN,NaN,NaN,NaN,NaN,5,August
19541,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",SWD,2024,5,2,9.0,0.0,9.0,0.0,0.0,0.0,9,August


In [144]:
swd_data = filter_graduation_data(all_swd_graduates, ['Graduation Year','Category','Duration'], cluster=True)
swd_data['Category Code'] = swd_data['Category'].astype('category').cat.codes
swd_data

,Graduation Year,Category,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,Category Code
0,2018,Not SWD,4,0,4021,947.0,56.0,828.0,63.0,2110.0,763.0,0
1,2018,Not SWD,4,1,25719,18453.0,2333.0,15153.0,967.0,4197.0,2156.0,0
2,2018,Not SWD,4,2,31167,26215.0,10616.0,15275.0,325.0,1939.0,483.0,0
3,2018,Not SWD,5,0,4437,1627.0,58.0,1421.0,148.0,1236.0,1257.0,0
4,2018,Not SWD,5,1,24753,18640.0,2289.0,15424.0,927.0,1310.0,3202.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2024,SWD,5,1,5109,4078.0,186.0,3768.0,124.0,160.0,690.0,1
122,2024,SWD,5,2,5211,4728.0,497.0,4150.0,81.0,103.0,303.0,1
123,2024,SWD,6,0,968,559.0,2.0,470.0,87.0,78.0,304.0,1
124,2024,SWD,6,1,5112,4059.0,382.0,3608.0,69.0,60.0,769.0,1


In [145]:
correlation_matrix(swd_data.drop('Category', axis=1))

### Population

In [330]:
swd_data_population = filter_population(swd_data)
swd_data_population

,Category,Cluster,# Total Cohort,# Grads,General Cohort %
0,Not SWD,0,79065,32028.0,77.011114
1,Not SWD,1,507699,400833.0,81.991276
2,Not SWD,2,639780,571944.0,86.462365
3,SWD,0,23602,9187.0,22.988886
4,SWD,1,111512,75789.0,18.008724
5,SWD,2,100172,84758.0,13.537635


In [340]:
fig = make_subplots(rows=1, cols=len(swd_data_population['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_population['Cluster'].unique():
    cluster_row = swd_data_population[swd_data_population['Cluster'] == cluster]
    cluster_index = swd_data_population['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Category'], y=category_row['General Cohort %'], marker=dict(color=colors[category_index]), showlegend=False), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0,swd_data_population['General Cohort %'].max()], title_text='Distibution %')


fig.update_layout(title_text='Distribution of Population Per Cohort Based on Disability Status')
fig.show()

### Graduation Rate

In [335]:
swd_data_graduation_rate = filter_graduation_rate(swd_data)
swd_data_graduation_rate

,Category,Graduation Year,Cluster,# Total Cohort,# Grads,Grad %
0,Not SWD,2018,0,13293,4775.0,35.921162
1,Not SWD,2018,1,75877,56388.0,74.315010
2,Not SWD,2018,2,91847,78837.0,85.835139
3,Not SWD,2019,0,12981,4643.0,35.767660
4,Not SWD,2019,1,75386,57049.0,75.675855
5,Not SWD,2019,2,91711,80435.0,87.704855
6,Not SWD,2020,0,12766,4898.0,38.367539
7,Not SWD,2020,1,75365,58782.0,77.996417
8,Not SWD,2020,2,91763,81278.0,88.573826
9,Not SWD,2021,0,11507,4875.0,42.365517


In [339]:
fig = make_subplots(rows=1, cols=len(swd_data_graduation_rate['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_graduation_rate['Cluster'].unique():
    cluster_row = swd_data_graduation_rate[swd_data_graduation_rate['Cluster'] == cluster]
    cluster_index = swd_data_graduation_rate['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Scatter(x=category_row['Graduation Year'], y=category_row['Grad %'], name=category, line=dict(color=colors[category_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[swd_data_graduation_rate['Grad %'].min(), swd_data_graduation_rate['Grad %'].max()], title_text='Graduation %')

for i in range(len(fig.data) - len(swd_data_graduation_rate['Category'].unique())):
    fig.data[i].showlegend=False

fig.update_layout(title_text='Graduation Rate from 2018 to 2024 in Every Cluster Based on Disability Status')

fig.show()

### Diploma Type

In [341]:
swd_data_diploma_type = filter_diploma_type(swd_data)
swd_data_diploma_type

,Category,Cluster,# Advanced Regents,# Regents without Advanced,# Local,# Grads,Advanced Regents %,Regents without Advanced %,Local %
0,Not SWD,0,1170.0,29370.0,1493.0,32028.0,3.653054,91.701012,4.661546
1,Not SWD,1,83328.0,306951.0,10601.0,400833.0,20.788708,76.578276,2.644742
2,Not SWD,2,279345.0,288932.0,3791.0,571944.0,48.841320,50.517533,0.662827
3,SWD,0,13.0,5571.0,3605.0,9187.0,0.141504,60.640035,39.240231
4,SWD,1,3608.0,52400.0,19795.0,75789.0,4.760585,69.139321,26.118566
5,SWD,2,10568.0,59809.0,14393.0,84758.0,12.468440,70.564430,16.981288


In [345]:
fig = make_subplots(rows=1, cols=len(swd_data_diploma_type['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_diploma_type['Cluster'].unique():
    cluster_row = swd_data_diploma_type[swd_data_diploma_type['Cluster'] == cluster]
    cluster_index = swd_data_diploma_type['Cluster'].unique().tolist().index(cluster)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Advanced Regents %'], name='Advanced Regents Diploma', marker=dict(color=colors[0])), row=1, col=cluster_index+1)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Regents without Advanced %'], name='Regular Regents Diploma', marker=dict(color=colors[1])), row=1, col=cluster_index+1)
    fig.add_trace(go.Bar(x=cluster_row['Category'], y=cluster_row['Local %'], name='Local Diploma', marker=dict(color=colors[2])), row=1, col=cluster_index+1)
    fig.update_yaxes(range=[0, max(swd_data_diploma_type[['Advanced Regents %', 'Regents without Advanced %', 'Local %']].max(axis=1))], title_text='Distribution %')

for i in range(len(fig.data) - 3):
    fig.data[i].showlegend=False

fig.update_layout(title_text='Distribution of Graduates Per Disability Status in Every Cluster Based on the Type of Diploma')
fig.show()

### Dropouts

In [346]:
swd_data_dropout = filter_dropout(swd_data)
swd_data_dropout

,Category,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Not SWD,0,19376.0,79065,32028.0,47037.0,41.193103
1,Not SWD,1,49474.0,507699,400833.0,106866.0,46.295361
2,Not SWD,2,12732.0,639780,571944.0,67836.0,18.768795
3,SWD,0,5487.0,23602,9187.0,14415.0,38.064516
4,SWD,1,20724.0,111512,75789.0,35723.0,58.013045
5,SWD,2,7212.0,100172,84758.0,15414.0,46.788634


In [353]:
fig = make_subplots(rows=1, cols=len(swd_data_dropout['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_dropout['Cluster'].unique():
    cluster_row =  swd_data_dropout[swd_data_dropout['Cluster'] == cluster]
    cluster_index = swd_data_dropout['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Category'], y=category_row['Dropout %'], marker=dict(color=colors[category_index]), showlegend=False), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, swd_data_dropout['Dropout %'].max()], title_text='Dropout %')

fig.show()

In [355]:
swd_data_dropout_per_year = filter_dropout_per_year(swd_data)
swd_data_dropout_per_year

,Category,Graduation Year,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Not SWD,2018,0,3677.0,13293,4775.0,8518.0,43.167410
1,Not SWD,2018,1,9059.0,75877,56388.0,19489.0,46.482631
2,Not SWD,2018,2,2056.0,91847,78837.0,13010.0,15.803228
3,Not SWD,2019,0,3638.0,12981,4643.0,8338.0,43.631566
4,Not SWD,2019,1,9058.0,75386,57049.0,18337.0,49.397393
5,Not SWD,2019,2,2265.0,91711,80435.0,11276.0,20.086910
6,Not SWD,2020,0,2859.0,12766,4898.0,7868.0,36.337062
7,Not SWD,2020,1,7941.0,75365,58782.0,16583.0,47.886390
8,Not SWD,2020,2,1962.0,91763,81278.0,10485.0,18.712446
9,Not SWD,2021,0,2167.0,11507,4875.0,6632.0,32.674910


In [363]:
fig = make_subplots(rows=1, cols=len(swd_data_dropout_per_year['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_dropout_per_year['Cluster'].unique():
    cluster_row = swd_data_dropout_per_year[swd_data_dropout_per_year['Cluster'] ==  cluster]
    cluster_index = swd_data_dropout_per_year['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Scatter(x=category_row['Graduation Year'], y=category_row['Dropout %'], name=category, line=dict(color=colors[category_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[swd_data_dropout_per_year['Dropout %'].min(), swd_data_dropout_per_year['Dropout %'].max()], title_text='Dropout %')
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text='Dropout Rates per Year for Each Cohort According to Disability Status')

for i in range(len(fig.data) - len(swd_data_dropout_per_year['Category'].unique())):
    fig.data[i].showlegend=False

fig.show()

In [364]:
swd_data_dropout_per_duration = filter_dropout_per_duration(swd_data)
swd_data_dropout_per_duration

,Category,Duration,Cluster,# Dropout_x,# Dropout_y,Dropout %
0,Not SWD,4,0,3703.0,19376.0,19.111272
1,Not SWD,5,0,6811.0,19376.0,35.151734
2,Not SWD,6,0,8862.0,19376.0,45.736994
3,Not SWD,4,1,11862.0,49474.0,23.976230
4,Not SWD,5,1,17633.0,49474.0,35.640943
5,Not SWD,6,1,19979.0,49474.0,40.382827
6,Not SWD,4,2,2840.0,12732.0,22.306001
7,Not SWD,5,2,4812.0,12732.0,37.794533
8,Not SWD,6,2,5080.0,12732.0,39.899466
9,SWD,4,0,1029.0,5487.0,18.753417


In [371]:
fig = make_subplots(rows=1, cols=len(swd_data_dropout_per_duration['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_dropout_per_duration['Cluster'].unique():
    cluster_row = swd_data_dropout_per_duration[swd_data_dropout_per_duration['Cluster'] ==  cluster]
    cluster_index = swd_data_dropout_per_duration['Cluster'].unique().tolist().index(cluster)
    for duration in cluster_row['Duration'].unique():
        duration_row = cluster_row[cluster_row['Duration'] == duration]
        duration_index = cluster_row['Duration'].unique().tolist().index(duration)
        fig.add_trace(go.Bar(x=duration_row['Category'], y=duration_row['Dropout %'], name=f'{duration} Years', marker=dict(color=colors[duration_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, swd_data_dropout_per_duration['Dropout %'].max()], title_text='Distribution %')

fig.update_layout(title_text='Distribution of Dropouts in Every Cluster per Duration and Disability Status')

for i in range(len(fig.data) - len(swd_data_dropout_per_duration['Duration'].unique())):
    fig.data[i].showlegend=False

fig.show()

### Test Results

In [115]:
swd_test_results

,School Name,Cluster,Year,Category,Regents Exam,Total Tested,Number Scoring 65 or Above,Regents Subject
0,A-TECH HIGH SCHOOL,1,2015,Non-SWD,Algebra2/Trigonometry,7,0.0,Mathematics
1,A-TECH HIGH SCHOOL,1,2015,SWD,Common Core Algebra,12,0.0,Mathematics
2,A-TECH HIGH SCHOOL,1,2015,Non-SWD,Common Core Algebra,29,5.0,Mathematics
3,A-TECH HIGH SCHOOL,1,2015,SWD,Common Core English,44,7.0,English
4,A-TECH HIGH SCHOOL,1,2015,Non-SWD,Common Core English,123,56.0,English
...,...,...,...,...,...,...,...,...
50212,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,SWD,Physical Settings/Earth Science,11,2.0,Science
50213,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,Non-SWD,Physical Settings/Earth Science,49,45.0,Science
50214,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,Non-SWD,Spanish,4,NaN,Languages Other Than English
50215,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",2,2023,SWD,US History and Government,5,NaN,Social Studies


In [117]:
swd_data_test_results = swd_test_results.groupby(['Category', 'Cluster', 'Regents Subject'])[['Number Scoring 65 or Above', 'Total Tested']].sum().reset_index()
swd_data_test_results['% Passed'] = swd_data_test_results['Number Scoring 65 or Above'] / swd_data_test_results['Total Tested'] * 100
swd_data_test_results

,Category,Cluster,Regents Subject,Number Scoring 65 or Above,Total Tested,% Passed
0,Non-SWD,0,English,9301.0,17529,53.060642
1,Non-SWD,0,Languages Other Than English,962.0,1696,56.721698
2,Non-SWD,0,Mathematics,5594.0,19966,28.017630
3,Non-SWD,0,Science,4595.0,15774,29.130214
4,Non-SWD,0,Social Studies,12126.0,28642,42.336429
5,Non-SWD,1,English,135311.0,195145,69.338697
6,Non-SWD,1,Languages Other Than English,29111.0,48978,59.436890
7,Non-SWD,1,Mathematics,174247.0,397375,43.849512
8,Non-SWD,1,Science,140658.0,293334,47.951482
9,Non-SWD,1,Social Studies,194849.0,307995,63.263689


In [118]:
fig = make_subplots(rows=1, cols=len(swd_data_test_results['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in swd_data_test_results['Cluster'].unique():
    cluster_row = swd_data_test_results[swd_data_test_results['Cluster'] == cluster]
    cluster_index = swd_data_test_results['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Regents Subject'], y=category_row['% Passed'], marker=dict(color=colors[category_index]), name=category), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, swd_data_test_results['% Passed'].max()], title_text='% Approved')

fig.update_layout(
    margin=dict(b=125),
    title_text='Rate of Students who Aproved Every Regents Subject in Every Cluster by Student Disability Status'
)

for i in range(len(fig.data) - len(swd_data_test_results['Category'].unique())):
    fig.data[i].showlegend=False
fig.show()

## Impact of Poverty Status

In [146]:
all_poverty_graduates = filter_data(poverty_graduation, columns=['# Grads','# Advanced Regents', '# Regents without Advanced', '# Local', '# Still Enrolled', '# Dropout'], cluster=True)
all_poverty_graduates

,School Name,Category,Graduation Year,Duration,Cluster,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,# Total Cohort,Graduation Month
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Econ Disadv,2018,4,2,NaN,NaN,NaN,NaN,NaN,NaN,30,August
1,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Econ Disadv,2018,5,2,NaN,NaN,NaN,NaN,NaN,NaN,30,August
2,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Econ Disadv,2018,6,2,40.0,6.0,29.0,5.0,0.0,1.0,43,June
3,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Econ Disadv,2019,4,2,NaN,NaN,NaN,NaN,NaN,NaN,36,August
4,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,Econ Disadv,2019,5,2,NaN,NaN,NaN,NaN,NaN,NaN,30,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19594,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Not Econ Disadv,2023,5,2,12.0,12.0,0.0,0.0,0.0,0.0,12,August
19595,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Not Econ Disadv,2023,6,2,18.0,15.0,3.0,0.0,1.0,0.0,19,June
19596,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Not Econ Disadv,2024,4,2,22.0,17.0,5.0,0.0,0.0,0.0,22,August
19597,"YOUNG WOMEN'S LEADERSHIP SCHOOL, QUEENS",Not Econ Disadv,2024,5,2,24.0,8.0,16.0,0.0,0.0,0.0,24,August


In [147]:
poverty_data = filter_graduation_data(all_poverty_graduates, ['Graduation Year','Category','Duration'], cluster=True)
poverty_data['Category Code'] = poverty_data['Category'].astype('category').cat.codes
poverty_data

,Graduation Year,Category,Duration,Cluster,# Total Cohort,# Grads,# Advanced Regents,# Regents without Advanced,# Local,# Still Enrolled,# Dropout,Category Code
0,2018,Econ Disadv,4,0,4121,803.0,20.0,648.0,135.0,2262.0,636.0,0
1,2018,Econ Disadv,4,1,25496,17423.0,1903.0,13333.0,2187.0,4787.0,2235.0,0
2,2018,Econ Disadv,4,2,24256,21071.0,7948.0,11893.0,1231.0,2160.0,499.0,0
3,2018,Econ Disadv,5,0,4280,1565.0,23.0,1184.0,358.0,1380.0,1192.0,0
4,2018,Econ Disadv,5,1,22904,17001.0,1910.0,12935.0,2156.0,1595.0,3260.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2024,Not Econ Disadv,5,1,5186,4462.0,965.0,3484.0,14.0,71.0,494.0,1
122,2024,Not Econ Disadv,5,2,11246,10961.0,5705.0,5231.0,25.0,53.0,162.0,1
123,2024,Not Econ Disadv,6,0,569,328.0,11.0,311.0,6.0,26.0,162.0,1
124,2024,Not Econ Disadv,6,1,5487,4718.0,1450.0,3247.0,21.0,21.0,585.0,1


### Correlation Matrix

In [148]:
correlation_matrix(poverty_data.drop('Category', axis=1))

In [377]:
poverty_data_population = filter_population(poverty_data)
poverty_data_population

,Category,Cluster,# Total Cohort,# Grads,General Cohort %
0,Econ Disadv,0,84068,32272.0,81.884150
1,Econ Disadv,1,494443,377688.0,79.850487
2,Econ Disadv,2,493755,453759.0,66.727977
3,Not Econ Disadv,0,18599,8078.0,18.115850
4,Not Econ Disadv,1,124768,99042.0,20.149513
5,Not Econ Disadv,2,246197,236022.0,33.272023


In [379]:
fig = make_subplots(rows=1, cols=len(poverty_data_population['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in poverty_data_population['Cluster'].unique():
    cluster_row = poverty_data_population[poverty_data_population['Cluster'] ==  cluster]
    cluster_index = poverty_data_population['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Category'], y=category_row['General Cohort %'], marker=dict(color=colors[category_index]), showlegend=False), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, poverty_data_population['General Cohort %'].max()], title_text='Distribution %')

fig.update_layout(title_text='Distribution of Population in Every Cluster Based on Poverty Status')

fig.show()

### Graduation Rate

In [385]:
poverty_data_graduation_rate = filter_graduation_rate(poverty_data)
poverty_data_graduation_rate

,Category,Graduation Year,Cluster,# Total Cohort,# Grads,Grad %
0,Econ Disadv,2018,0,12828,4443.0,34.635173
1,Econ Disadv,2018,1,70692,51434.0,72.757879
2,Econ Disadv,2018,2,67709,60946.0,90.011668
3,Econ Disadv,2019,0,12950,4777.0,36.888031
4,Econ Disadv,2019,1,72184,53753.0,74.466641
5,Econ Disadv,2019,2,69684,63173.0,90.656392
6,Econ Disadv,2020,0,13448,5402.0,40.169542
7,Econ Disadv,2020,1,73554,55636.0,75.639666
8,Econ Disadv,2020,2,70781,64894.0,91.682796
9,Econ Disadv,2021,0,12440,5254.0,42.234727


In [389]:
fig = make_subplots(rows=1, cols=len(poverty_data_graduation_rate['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in poverty_data_graduation_rate['Cluster'].unique():
    cluster_row = poverty_data_graduation_rate[poverty_data_graduation_rate['Cluster'] == cluster]
    cluster_index = poverty_data_graduation_rate['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Scatter(x=category_row['Graduation Year'], y=category_row['Grad %'], name=category, line=dict(color=colors[category_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[poverty_data_graduation_rate['Grad %'].min(), poverty_data_graduation_rate['Grad %'].max()], title_text='Graduation %')
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text='Graduation Rate from 2018 to 2024 in Every Cluster Based on Poverty Status')

for i in range(len(fig.data) - len(poverty_data_graduation_rate['Category'].unique())):
    fig.data[i].showlegend = False

fig.show()

### Dropouts

In [390]:
poverty_data_dropout =  filter_dropout(poverty_data)
poverty_data_dropout

,Category,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Econ Disadv,0,19389.0,84068,32272.0,51796.0,37.433393
1,Econ Disadv,1,52679.0,494443,377688.0,116755.0,45.119267
2,Econ Disadv,2,14409.0,493755,453759.0,39996.0,36.026103
3,Not Econ Disadv,0,5209.0,18599,8078.0,10521.0,49.510503
4,Not Econ Disadv,1,17410.0,124768,99042.0,25726.0,67.674726
5,Not Econ Disadv,2,5435.0,246197,236022.0,10175.0,53.415233


In [402]:
fig = make_subplots(rows=1, cols=len(poverty_data_dropout['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in poverty_data_dropout['Cluster'].unique():
    cluster_row = poverty_data_dropout[poverty_data_dropout['Cluster'] == cluster]
    cluster_index = poverty_data_dropout['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Bar(x=category_row['Category'], y=category_row['Dropout %'], marker=dict(color=colors[category_index]), showlegend=False), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0, poverty_data_dropout['Dropout %'].max()], title_text='Dropout %')

fig.update_layout(title_text='Dropout Rate of Non-Graduates in Every Cluster by Poverty Status')

fig.show()

In [397]:
poverty_data_dropout_per_year = filter_dropout_per_year(poverty_data)
poverty_data_dropout_per_year

,Category,Graduation Year,Cluster,# Dropout,# Total Cohort,# Grads,# No Grads,Dropout %
0,Econ Disadv,2018,0,3320.0,12828,4443.0,8385.0,39.594514
1,Econ Disadv,2018,1,9014.0,70692,51434.0,19258.0,46.806522
2,Econ Disadv,2018,2,2015.0,67709,60946.0,6763.0,29.794470
3,Econ Disadv,2019,0,3689.0,12950,4777.0,8173.0,45.136425
4,Econ Disadv,2019,1,9827.0,72184,53753.0,18431.0,53.317780
5,Econ Disadv,2019,2,2424.0,69684,63173.0,6511.0,37.229304
6,Econ Disadv,2020,0,3004.0,13448,5402.0,8046.0,37.335322
7,Econ Disadv,2020,1,8536.0,73554,55636.0,17918.0,47.639245
8,Econ Disadv,2020,2,2216.0,70781,64894.0,5887.0,37.642263
9,Econ Disadv,2021,0,2386.0,12440,5254.0,7186.0,33.203451


In [401]:
fig = make_subplots(rows=1, cols=len(poverty_data_dropout_per_year['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in poverty_data_dropout_per_year['Cluster'].unique():
    cluster_row = poverty_data_dropout_per_year[poverty_data_dropout_per_year['Cluster'] == cluster]
    cluster_index = poverty_data_dropout_per_year['Cluster'].unique().tolist().index(cluster)
    for category in cluster_row['Category'].unique():
        category_row = cluster_row[cluster_row['Category'] == category]
        category_index = cluster_row['Category'].unique().tolist().index(category)
        fig.add_trace(go.Scatter(x=category_row['Graduation Year'], y=category_row['Dropout %'], name=category, line=dict(color=colors[category_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[poverty_data_dropout_per_year['Dropout %'].min(), poverty_data_dropout_per_year['Dropout %'].max()], title_text='Dropout %')
        fig.update_xaxes(title_text='Year')

fig.update_layout(title_text='Dropout Rates from 2018 to 2024 in Every Cluster by Poverty Status')

for i in range(len(fig.data) - len(poverty_data_dropout['Category'].unique())):
    fig.data[i].showlegend=False

fig.show()

In [403]:
poverty_data_dropout_per_duration = filter_dropout_per_duration(poverty_data)
poverty_data_dropout_per_duration

,Category,Duration,Cluster,# Dropout_x,# Dropout_y,Dropout %
0,Econ Disadv,4,0,3463.0,19389.0,17.860643
1,Econ Disadv,5,0,6936.0,19389.0,35.772861
2,Econ Disadv,6,0,8990.0,19389.0,46.366496
3,Econ Disadv,4,1,12301.0,52679.0,23.350861
4,Econ Disadv,5,1,19044.0,52679.0,36.151028
5,Econ Disadv,6,1,21334.0,52679.0,40.498111
6,Econ Disadv,4,2,3315.0,14409.0,23.006454
7,Econ Disadv,5,2,5448.0,14409.0,37.809702
8,Econ Disadv,6,2,5646.0,14409.0,39.183843
9,Not Econ Disadv,4,0,922.0,5209.0,17.700134


In [406]:
fig = make_subplots(rows=1, cols=len(poverty_data_dropout_per_duration['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in poverty_data_dropout_per_duration['Cluster'].unique():
    cluster_row = poverty_data_dropout_per_duration[poverty_data_dropout_per_duration['Cluster'] == cluster]
    cluster_index = poverty_data_dropout_per_duration['Cluster'].unique().tolist().index(cluster)
    for duration in cluster_row['Duration'].unique():
        duration_row = cluster_row[cluster_row['Duration'] == duration]
        duration_index = cluster_row['Duration'].unique().tolist().index(duration)
        fig.add_trace(go.Bar(x=duration_row['Category'], y=duration_row['Dropout %'], name=f'{duration} Years', marker=dict(color=colors[duration_index])), row=1, col=cluster_index+1)
        fig.update_yaxes(range=[0,poverty_data_dropout_per_duration['Dropout %'].max()], title_text='Distribution %')

fig.update_layout(title_text='Distribution of Dropouts Based on Duration in Every Cluster by Poverty Status')

for i in range(len(fig.data) - len(poverty_data_dropout_per_duration['Duration'].unique())):
    fig.data[i].showlegend=False

fig.show()

## Predictions for the Next 5 Years

### General

In [232]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [234]:
general_data_population

,Graduation Year,# Total Cohort,# Grads,Grad %
0,2018,213338,172686.0,80.944792
1,2019,213275,175297.0,82.192943
2,2020,214042,180304.0,84.237673
3,2021,211698,182263.0,86.095759
4,2022,208741,182956.0,87.647372
5,2023,204843,181402.0,88.556602
6,2024,201475,178675.0,88.683459


In [235]:
x = general_data_population['Graduation Year'].values
y = general_data_population['Grad %'].values
np.corrcoef(x, y)[0,1]

0.9788900474312925

In [236]:
x = x.reshape(-1,1)
model = LinearRegression()
model.fit(x,y)

LinearRegression()

In [257]:
def get_future_years(df):
    future_years = []
    for i in range(1,6):
        future_years.append(df['Graduation Year'].max() + i)
    
    future_years = np.array(future_years)
    return future_years

In [258]:
future_years = get_future_years(general_data_population)
future_years

array([2025, 2026, 2027, 2028, 2029])

In [259]:
future_preds = model.predict(future_years.reshape(-1,1))
future_preds

array([91.10165984, 92.50712479, 93.91258974, 95.31805469, 96.72351964])

In [260]:
current_trend = model.predict(x)
current_trend

array([81.26340518, 82.66887013, 84.07433508, 85.47980003, 86.88526499,
       88.29072994, 89.69619489])

In [261]:
r2_score(y, current_trend)

0.9582257249600389

In [267]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=general_data_population['Graduation Year'], y=general_data_population['Grad %'], mode='markers', name='Current Data'))
fig.add_trace(go.Scatter(x=general_data_population['Graduation Year'], y=current_trend, mode='lines', name='Current Trend'))
fig.add_trace(go.Scatter(x=future_years, y=future_preds, mode='markers', marker=dict(symbol="x", size=10), name='Predictions'))
fig.update_layout(
    title="Graduation Rate Predictions for the Next 5 Years",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    xaxis=dict(
        tickvals=np.append(general_data_population['Graduation Year'].values,future_years), 
        ticktext=np.append(general_data_population['Graduation Year'].values,future_years)
    )
)

### Cluster

In [268]:
general_data_cluster_rate

,Cluster,Graduation Year,# Total Cohort,# Grads,Grad %
0,0,2018,16531,6148.0,37.190733
1,0,2019,16447,6224.0,37.842768
2,0,2020,16496,6795.0,41.191804
3,0,2021,15057,6770.0,44.962476
4,0,2022,13333,6214.0,46.606165
5,0,2023,12250,5578.0,45.534694
6,0,2024,12553,5896.0,46.968852
7,1,2018,92602,69497.0,75.049135
8,1,2019,91860,71002.0,77.293708
9,1,2020,91519,73368.0,80.166960


In [281]:
fig = make_subplots(rows=1, cols=len(general_data_cluster_rate['Cluster'].unique()), subplot_titles=('Cluster 0', 'Cluster 1', 'Cluster 2'))

for cluster in general_data_cluster_rate['Cluster'].unique():
    cluster_row = general_data_cluster_rate[general_data_cluster_rate['Cluster'] == cluster]
    cluster_index = general_data_cluster_rate['Cluster'].unique().tolist().index(cluster)
    x = cluster_row['Graduation Year'].values
    y = cluster_row['Grad %'].values
    
    x = x.reshape(-1,1)
    model = LinearRegression()
    model.fit(x,y)

    future_years = get_future_years(general_data_population)
    future_preds = model.predict(future_years.reshape(-1,1))
    current_trend = model.predict(x)
    fig.add_trace(go.Scatter(x=cluster_row['Graduation Year'], y=cluster_row['Grad %'], mode='markers', name='Current Data', marker=dict(color=colors[cluster_index])), row=1, col=cluster_index+1)
    fig.add_trace(go.Scatter(x=cluster_row['Graduation Year'], y=current_trend, mode='lines', name='Current Trend', line=dict(color=colors[cluster_index])), row=1, col=cluster_index+1)
    fig.add_trace(go.Scatter(x=future_years, y=future_preds, mode='markers', marker=dict(symbol="x", size=10, color=colors[cluster_index]), name='Predictions'), row=1, col=cluster_index+1)
    fig.update_yaxes(range=[general_data_cluster_rate['Grad %'].min(), 100], title_text='Graduation %')
    fig.update_xaxes(dtick=2, title_text='Year')


fig.update_layout(title="Graduation Rate Predictions for the Next 5 Years for Every Cluster")

for i in range(len(fig.data) - len(general_data_cluster_rate['Cluster'].unique())):
    fig.data[i].showlegend=False
fig.show()